# Preprocessing

In [1]:
import numpy as np
import pandas as pd
import os
import pickle
import utm
import random

In [2]:
import prepostprocessing.cleaning as cleaning
import prepostprocessing.pre_processing as preproc

from qapf import qapf
from qapf import cipw

In [3]:
from sklearn.preprocessing import StandardScaler

In [4]:
# Load jupyter extension to reload packages before executing user code.
# https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html
%load_ext autoreload
# Reload all packages (except those excluded by %aimport) every time before executing the Python code typed.
%autoreload 2

## Mineralogy

**To do**
* ~~Clean last points in Excel file while using "sum" as check~~

In [5]:
mineralogy = pd.read_excel("../_CLEANED/Vistelius_data_cleaned_all.xlsx", index_col=0)

mineralogyP = pd.read_excel("../_CLEANED/Vistelius_data_cleaned_all.xlsx", index_col=0)

### Check for wrong entries

In [6]:
np.isclose(mineralogy.loc[:, :"oth"].sum(axis=1), mineralogy.loc[:, "sum"])

array([ True,  True,  True, ...,  True,  True,  True])

In [7]:
wrong_sum = mineralogy.loc[~np.isclose(mineralogy.loc[:, :"oth"].sum(axis=1), mineralogy.loc[:, "sum"]), :]

In [8]:
# Check to see if any remaining incorrect lines are present
wrong_sum

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O,P2O5,l.i.,oth,sum,hs


In [9]:
mineralogy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4659 entries, 1 to 4659
Data columns (total 15 columns):
SiO2     4659 non-null float64
TiO2     4626 non-null float64
Al2O3    4659 non-null float64
Fe2O3    4657 non-null float64
FeO      4659 non-null float64
MnO      4544 non-null float64
MgO      4658 non-null float64
CaO      4659 non-null float64
Na2O     4659 non-null float64
K2O      4659 non-null float64
P2O5     3834 non-null float64
l.i.     4659 non-null float64
oth      874 non-null float64
sum      4659 non-null float64
hs       2240 non-null float64
dtypes: float64(15)
memory usage: 582.4 KB


In [10]:
wrong_sum.loc[:, :"oth"].sum(axis=1)

Series([], dtype: float64)

In [11]:
mineralogy = preproc.replace_nan(mineralogy, 0)
mineralogyP = preproc.replace_nan(mineralogyP, 0)

In [12]:
mineralogy['oth'] = mineralogy['P2O5'] + mineralogy['l.i.'] + mineralogy['oth']
mineralogy.drop(["l.i.", "P2O5", ], axis = 1, inplace = True)
mineralogyP.drop(["l.i.", ], axis = 1, inplace = True)

In [13]:
mineralogyP.head()

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O,P2O5,oth,sum,hs
1,80.80,0.04,10.16,0.61,1.72,0.00,0.40,0.55,2.00,3.59,0.00,0.00,100.22,0.00
2,80.00,0.10,10.10,0.17,0.56,0.02,0.40,0.35,2.30,5.10,0.05,0.00,99.65,0.00
3,79.92,0.05,9.89,0.16,1.73,0.02,0.12,0.14,0.75,6.15,0.00,0.08,100.03,0.30
4,79.65,0.04,9.64,1.15,0.75,0.10,0.45,0.67,3.71,4.25,0.00,0.00,100.67,0.01
5,79.18,0.08,10.24,0.64,2.60,0.04,0.05,1.25,1.52,3.08,0.01,0.16,100.60,0.28


### Cleaning
**To do**
* ~~Replace zero values~~


In [14]:
# Would not do this this way since it becomes less clear what the variable means
# You should also replace 'minralogy' in all remaining cells by 'x' if you would want to do this
# x = mineralogy

In [15]:
# Replace zero values
mineralogy = preproc.replace_zero(mineralogy, 0.01)
mineralogyP = preproc.replace_zero(mineralogyP, 0.01)

* ~~Normalize~~

In [16]:
# Normalize specific columns
mineralogy.loc[:, :"oth"] = preproc.normalize(mineralogy.loc[:, :"oth"])# , total=mineralogy['sum'])
mineralogyP.loc[:, :"oth"] = preproc.normalize(mineralogyP.loc[:, :"oth"])# , total=mineralogy['sum'])

In [17]:
# Renew 'sum' column to reflect changes applied during cleaning
mineralogy["sum"] = mineralogy.loc[:, :"oth"].sum(axis=1)
mineralogyP["sum"] = mineralogyP.loc[:, :"oth"].sum(axis=1)

In [18]:
# Check that sum of all variables + 'sum' == 200
assert all(np.isclose(mineralogy.loc[:, :'sum'].sum(axis=1), 200.0))
assert all(np.isclose(mineralogyP.loc[:, :'sum'].sum(axis=1), 200.0))

### centred log-ratio (clr) transformation

In [19]:
mineralogy1 = mineralogy.drop(columns=['oth', 'hs', 'sum'])
mineralogyP1 = mineralogyP.drop(columns=['oth', 'hs', 'sum'])

In [20]:
mineralogy_clr1 = preproc.clr(mineralogy1)
mineralogy_clr1.head()

mineralogy_clrP1 = preproc.clr(mineralogyP1)
mineralogy_clrP1.head()

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O,P2O5
1,4.865939,-2.744914,2.792420,-0.020334,1.016286,-4.131208,-0.442329,-0.123875,1.167109,1.752114,-4.131208
2,4.779437,-1.905174,2.709946,-1.374546,-0.182408,-3.514612,-0.518880,-0.652412,1.230320,2.026651,-2.598322
3,5.170339,-2.206419,3.080837,-1.043269,1.337434,-3.122710,-1.330951,-1.176800,0.501631,2.605765,-3.815857
4,4.566008,-3.030510,2.454287,0.328128,-0.099316,-2.114219,-0.610142,-0.212112,1.499398,1.635285,-4.416805
5,4.769130,-2.128322,2.723708,-0.048880,1.352918,-2.821469,-2.598326,0.620550,0.816117,1.522336,-4.207763


### Principal Component Analysis (PCA)

In [21]:
mineralogy_pca1 = preproc.pca(mineralogy_clr1)
preproc.pca_variance(mineralogy_pca1)

6 PCA components  out of 10 components with variance sum 0.9653845247577469 needed for obtaining sum of variance > 0.95


array([4.36495556e-01, 2.24309391e-01, 1.14379085e-01, 7.92733011e-02,
       6.47910351e-02, 4.61361562e-02, 2.56884815e-02, 7.25584036e-03,
       1.67115334e-03, 2.35922349e-32])

In [22]:
mineralogy_pca_df1 = preproc.create_pca_df(mineralogy_pca1, mineralogy_clr1)

In [23]:
mineralogy_pca_df1

,PC01,PC02,PC03,PC04,PC05,PC06,PC07,PC08,PC09,PC10
1,-0.989677,0.063658,-1.077309,0.949276,0.196830,-1.052766,-0.196021,0.313477,0.228518,-3.805685e-16
2,-1.170683,0.906362,-0.457936,0.315585,0.836318,0.547351,-0.165217,0.278892,0.207743,2.205780e-16
3,-1.952305,1.084163,0.126341,-0.059648,0.787068,-1.028997,-0.907513,0.941895,0.141972,-5.419556e-17
4,-1.265500,-0.665278,0.792680,1.554351,0.290969,0.026927,-0.145361,-0.158186,0.272325,-2.268773e-16
5,-1.401423,0.090158,0.667637,-0.203788,-1.529623,-1.127234,-0.414450,0.462700,0.259138,-2.570066e-16
...,...,...,...,...,...,...,...,...,...,...
4655,1.969576,-1.154415,0.734286,0.877843,-0.311676,-0.413926,0.060452,-0.189310,-0.243017,1.875335e-16
4656,2.765959,-0.679754,0.061989,-0.052525,-0.149849,-0.699663,0.034609,0.112326,-0.131631,2.651447e-18
4657,2.610638,-1.211222,0.315811,0.056660,-0.001314,-0.147221,-0.224187,0.059057,-0.149794,-8.143807e-16
4658,2.415960,-0.786335,0.173592,-0.202025,-0.649209,-0.020704,0.652916,0.082545,-0.300729,2.716508e-17


In [24]:
preproc.save_obj(mineralogy_pca_df1, "mineralogy_pca_df1")

In [25]:
metadata = pd.read_excel("../_INTERPOLATION/coordinates_full_data.xlsx", index_col=0, usecols=[0, 23, 24, 25, 26, 27, 28])

In [26]:
preproc.save_obj(metadata, "metadata")

In [27]:
preproc.save_obj(mineralogy_clr1, "mineralogy_clr1")

----

### making time and coordinate classes

In [28]:
time = pd.read_excel("../_RESULTS/time_fix.xlsx", index_col=0, usecols=[0, 9])
coordinates = pd.read_excel("../_INTERPOLATION/coordinates_UTM.xlsx", index_col=0, usecols=[0, 1, 2])

In [29]:
classification_renaming_dict = {"Tr" : "1",
                                "Tr-J" : "2",
                                "J" : "3",
                                "K" : "4",
                                "Pg" : "5",
                                "Mz" : "6"}   

In [30]:
time['time'] = time['time'].replace(classification_renaming_dict)

In [31]:
coordinates

,Y_UTM,X_UTM
1,7.880625e+05,4.936585e+06
2,-5.134927e+05,6.119173e+06
3,1.544245e+06,7.110278e+06
4,1.083775e+06,6.877413e+06
5,1.641085e+06,7.925964e+06
...,...,...
4655,1.756745e+06,7.702655e+06
4656,6.586139e+05,5.200781e+06
4657,6.880542e+05,6.455451e+06
4658,1.177291e+06,6.811066e+06


# Feature engineering: Frost

## Fe_Number

In [32]:
mineralogy_clr1

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O
1,4.452818,-3.158035,2.379300,-0.433455,0.603165,-4.544329,-0.855450,-0.536996,0.753988,1.338993
2,4.519605,-2.165007,2.450114,-1.634378,-0.442240,-3.774445,-0.778712,-0.912244,0.970488,1.766819
3,4.788753,-2.588005,2.699251,-1.424854,0.955849,-3.504296,-1.712536,-1.558386,0.120045,2.224179
4,4.124327,-3.472191,2.012606,-0.113553,-0.540997,-2.555900,-1.051822,-0.653792,1.057717,1.193604
5,4.348354,-2.549098,2.302932,-0.469657,0.932142,-3.242245,-3.019102,0.199774,0.395341,1.101560
...,...,...,...,...,...,...,...,...,...,...
4655,2.635045,-2.240153,1.668789,0.110001,0.304876,-2.812672,0.033822,0.783394,0.117309,-0.600410
4656,2.640723,-1.289140,1.587809,-0.329790,0.906973,-3.591725,0.334201,0.846209,-0.310814,-0.794444
4657,2.529955,-1.250274,1.502001,0.047427,0.262462,-3.222617,0.374695,0.744131,-0.404219,-0.583560
4658,2.737015,-1.059568,1.909712,-0.357610,0.248526,-3.484370,-0.177484,1.059988,0.219398,-1.095607


In [33]:
Fe_num1 = pd.read_excel("../_CLEANED/Vistelius_data_cleaned_all.xlsx", index_col=0, usecols = 0)

C:\Users\Sebastian\Anaconda3\envs\main\lib\site-packages\pandas\io\excel\_base.py:445: FutureWarning: Passing in an integer for `usecols` has been deprecated. Please pass in a list of int from 0 to `usecols` inclusive instead.
  usecols = _maybe_convert_usecols(usecols)


In [34]:
Fe_num1['Fe'] = mineralogy_clr1['FeO'] + mineralogy_clr1['Fe2O3'] 
Fe_num1['Fe+Mg']= mineralogy_clr1['MgO'] +mineralogy_clr1['FeO'] + mineralogy_clr1['Fe2O3']
Fe_num1['SiO2'] = mineralogy_clr1['SiO2']

In [35]:
Fe_x1 = Fe_num1.iloc[:,:1]
Fe_x2 = Fe_num1.iloc[:,1:2]
Fe_x3 = Fe_num1.iloc[:,2:3]

#Fe_x4 = time
#Fe_x5 = coordinates.iloc[:,:1]
#Fe_x6 = coordinates.iloc[:,1:2]

In [36]:
#Fe_x4

In [37]:
Fe_num1

,Fe,Fe+Mg,SiO2
1,0.169710,-0.685739,4.452818
2,-2.076618,-2.855331,4.519605
3,-0.469006,-2.181542,4.788753
4,-0.654550,-1.706372,4.124327
5,0.462485,-2.556617,4.348354
...,...,...,...
4655,0.414876,0.448698,2.635045
4656,0.577183,0.911383,2.640723
4657,0.309888,0.684583,2.529955
4658,-0.109084,-0.286567,2.737015


In [38]:
scaler1 = StandardScaler().fit(Fe_x1)
scaler2 = StandardScaler().fit(Fe_x2)
scaler3 = StandardScaler().fit(Fe_x3)

#scaler4 = StandardScaler().fit(Fe_x4)
#scaler5 = StandardScaler().fit(Fe_x5)
#scaler6 = StandardScaler().fit(Fe_x6)

In [39]:
Fe_x1_scaled = scaler1.transform(Fe_x1)
Fe_x2_scaled = scaler2.transform(Fe_x2)
Fe_x3_scaled = scaler3.transform(Fe_x3)

#Fe_x4_scaled = scaler4.transform(Fe_x4)
#Fe_x5_scaled = scaler5.transform(Fe_x5)
#Fe_x6_scaled = scaler6.transform(Fe_x6)

In [40]:
Fe_x1_df = pd.DataFrame(data =Fe_x1_scaled,columns=['Fe'])
Fe_x2_df = pd.DataFrame(data =Fe_x2_scaled,columns=['Fe+Mg'])
Fe_x3_df = pd.DataFrame(data =Fe_x3_scaled,columns=['SiO2'])

#Fe_x4_df = pd.DataFrame(data =Fe_x4_scaled,columns=['time'])
#Fe_x5_df = pd.DataFrame(data =Fe_x5_scaled,columns=['Y_UTM'])
#Fe_x6_df = pd.DataFrame(data =Fe_x6_scaled,columns=['X_UTM'])

In [41]:
Fe_num = pd.concat([Fe_num1.iloc[:,:0], Fe_x1_df, Fe_x2_df], axis=1, sort=True)
Fe_num.index = np.arange(1, len(Fe_num) + 1)
Fe_num = Fe_num.dropna()

In [42]:
Fe_num

,Fe,Fe+Mg
1,1.324521,1.004741
2,-1.884229,-1.104543
3,0.412153,-0.449484
4,0.147114,0.012478
5,1.742733,-0.814132
...,...,...
4655,1.674727,2.107645
4656,1.906572,2.557469
4657,1.524758,2.336973
4658,0.926281,1.392818


In [43]:
preproc.save_obj(Fe_num, "Fe_num")

In [44]:
Fe_num

,Fe,Fe+Mg
1,1.324521,1.004741
2,-1.884229,-1.104543
3,0.412153,-0.449484
4,0.147114,0.012478
5,1.742733,-0.814132
...,...,...
4655,1.674727,2.107645
4656,1.906572,2.557469
4657,1.524758,2.336973
4658,0.926281,1.392818


In [45]:
Fe_num_pca = preproc.pca(Fe_num)
preproc.pca_variance(Fe_num_pca)

2 PCA components  out of 2 components with variance sum 1.0 needed for obtaining sum of variance > 0.95


array([0.83456435, 0.16543565])

In [46]:
Fe_num_pca_df = preproc.create_pca_df(Fe_num_pca, Fe_num)

In [47]:
preproc.save_obj(Fe_num_pca_df, "Fe_num_pca_df")

## MALI

In [48]:
MALI1 = pd.read_excel("../_CLEANED/Vistelius_data_cleaned_all.xlsx", index_col=0, usecols = 0)

C:\Users\Sebastian\Anaconda3\envs\main\lib\site-packages\pandas\io\excel\_base.py:445: FutureWarning: Passing in an integer for `usecols` has been deprecated. Please pass in a list of int from 0 to `usecols` inclusive instead.
  usecols = _maybe_convert_usecols(usecols)


In [49]:
MALI1['Na2O'] = mineralogy_clr1['Na2O']
MALI1['K2O'] = mineralogy_clr1['K2O']
MALI1['CaO'] = mineralogy_clr1['CaO']
MALI1['SiO2'] = mineralogy_clr1['SiO2']


In [50]:
MALI_x1 = MALI1.iloc[:,:1]
MALI_x2 = MALI1.iloc[:,1:2]
MALI_x3 = MALI1.iloc[:,2:3]
MALI_x4 = MALI1.iloc[:,3:4]

MALI_x5 = time

In [51]:
MALI1

,Na2O,K2O,CaO,SiO2
1,0.753988,1.338993,-0.536996,4.452818
2,0.970488,1.766819,-0.912244,4.519605
3,0.120045,2.224179,-1.558386,4.788753
4,1.057717,1.193604,-0.653792,4.124327
5,0.395341,1.101560,0.199774,4.348354
...,...,...,...,...
4655,0.117309,-0.600410,0.783394,2.635045
4656,-0.310814,-0.794444,0.846209,2.640723
4657,-0.404219,-0.583560,0.744131,2.529955
4658,0.219398,-1.095607,1.059988,2.737015


In [52]:
scaler1 = StandardScaler().fit(MALI_x1)
scaler2 = StandardScaler().fit(MALI_x2)
scaler3 = StandardScaler().fit(MALI_x3)
scaler4 = StandardScaler().fit(MALI_x4)

scaler5 = StandardScaler().fit(MALI_x5)

In [53]:
MALI_x1_scaled = scaler1.transform(MALI_x1)
MALI_x2_scaled = scaler2.transform(MALI_x2)
MALI_x3_scaled = scaler3.transform(MALI_x3)
MALI_x4_scaled = scaler4.transform(MALI_x4)

MALI_x5_scaled = scaler5.transform(MALI_x5)

In [54]:
MALI_x1_scaled = pd.DataFrame(data =MALI_x1_scaled,columns=['Na2O'])
MALI_x2_scaled = pd.DataFrame(data =MALI_x2_scaled,columns=['K2O'])
MALI_x3_scaled = pd.DataFrame(data =MALI_x3_scaled,columns=['CaO'])
MALI_x4_scaled = pd.DataFrame(data =MALI_x4_scaled,columns=['SiO2'])

MALI_x5_scaled = pd.DataFrame(data =MALI_x5_scaled,columns=['time'])

In [55]:
MALI = pd.concat([MALI1.iloc[:,:0], MALI_x1_scaled, MALI_x2_scaled, MALI_x3_scaled], axis=1, sort=True)
MALI.index = np.arange(1, len(MALI) + 1)
MALI = MALI.dropna()

In [56]:
MALI

,Na2O,K2O,CaO
1,0.122888,0.940794,-0.636242
2,0.628920,1.627214,-1.229407
3,-1.358851,2.361020,-2.250782
4,0.832804,0.707527,-0.820865
5,-0.715393,0.559848,0.528392
...,...,...,...
4655,-1.365247,-2.170856,1.450937
4656,-2.365914,-2.482172,1.550231
4657,-2.584233,-2.143822,1.388872
4658,-1.126630,-2.965370,1.888157


In [57]:
preproc.save_obj(MALI, "MALI")

In [58]:
MALI_pca = preproc.pca(MALI)
preproc.pca_variance(MALI_pca)

3 PCA components  out of 3 components with variance sum 1.0 needed for obtaining sum of variance > 0.95


array([0.75985197, 0.13716298, 0.10298505])

In [59]:
MALI_pca_df = preproc.create_pca_df(MALI_pca, MALI)

In [60]:
preproc.save_obj(MALI_pca_df, "MALI_pca_df")

-----

## ASI

In [61]:
ASI = pd.read_excel("../_CLEANED/Vistelius_data_cleaned_all.xlsx", index_col=0, usecols = 0)

C:\Users\Sebastian\Anaconda3\envs\main\lib\site-packages\pandas\io\excel\_base.py:445: FutureWarning: Passing in an integer for `usecols` has been deprecated. Please pass in a list of int from 0 to `usecols` inclusive instead.
  usecols = _maybe_convert_usecols(usecols)


In [62]:
ASI['Al'] = mineralogy_clrP1['Al2O3']
ASI['Ca-P+Na+K'] = mineralogy_clrP1['Al2O3'] - (1.67* mineralogy_clrP1['P2O5']) + mineralogy_clrP1['Na2O'] + mineralogy_clrP1['K2O']


In [63]:
ASI

,Al,Ca-P+Na+K
1,2.792420,12.610761
2,2.709946,10.306114
3,3.080837,12.560714
4,2.454287,12.965033
5,2.723708,12.089127
...,...,...
4655,1.834805,4.456205
4656,1.827464,5.203770
4657,1.697006,4.355827
4658,2.087740,4.540664


In [64]:
preproc.save_obj(ASI, "ASI")

In [65]:
ASI_pca = preproc.pca(ASI)
preproc.pca_variance(ASI_pca)

1 PCA components  out of 2 components with variance sum 0.9924299020615133 needed for obtaining sum of variance > 0.95


array([0.9924299, 0.0075701])

In [66]:
ASI_pca_df = preproc.create_pca_df(ASI_pca, ASI)

In [67]:
preproc.save_obj(ASI_pca_df, "ASI_pca_df")

------

# incorporating coordinates and timeclass

In [68]:
mineralogy_inc_pca_df = mineralogy_pca_df1.iloc[:, 0: 7]
time = pd.read_excel("../_RESULTS/time_fix.xlsx", index_col=0, usecols=[0, 9])
coordinates = pd.read_excel("../_INTERPOLATION/coordinates_UTM.xlsx", index_col=0, usecols=[0, 1, 2])

**remark** : make sure to use the right coordinate system

In [69]:
coordinates

,Y_UTM,X_UTM
1,7.880625e+05,4.936585e+06
2,-5.134927e+05,6.119173e+06
3,1.544245e+06,7.110278e+06
4,1.083775e+06,6.877413e+06
5,1.641085e+06,7.925964e+06
...,...,...
4655,1.756745e+06,7.702655e+06
4656,6.586139e+05,5.200781e+06
4657,6.880542e+05,6.455451e+06
4658,1.177291e+06,6.811066e+06


In [70]:
classification_renaming_dict = {"Tr" : "1",
                                "Tr-J" : "2",
                                "J" : "3",
                                "K" : "4",
                                "Pg" : "5",
                                "Mz" : "6"}                         

In [71]:
time['time'] = time['time'].replace(classification_renaming_dict)

In [72]:
x8 = time

In [73]:
mineralogy_inc_pca_df

,PC01,PC02,PC03,PC04,PC05,PC06,PC07
1,-0.989677,0.063658,-1.077309,0.949276,0.196830,-1.052766,-0.196021
2,-1.170683,0.906362,-0.457936,0.315585,0.836318,0.547351,-0.165217
3,-1.952305,1.084163,0.126341,-0.059648,0.787068,-1.028997,-0.907513
4,-1.265500,-0.665278,0.792680,1.554351,0.290969,0.026927,-0.145361
5,-1.401423,0.090158,0.667637,-0.203788,-1.529623,-1.127234,-0.414450
...,...,...,...,...,...,...,...
4655,1.969576,-1.154415,0.734286,0.877843,-0.311676,-0.413926,0.060452
4656,2.765959,-0.679754,0.061989,-0.052525,-0.149849,-0.699663,0.034609
4657,2.610638,-1.211222,0.315811,0.056660,-0.001314,-0.147221,-0.224187
4658,2.415960,-0.786335,0.173592,-0.202025,-0.649209,-0.020704,0.652916


In [74]:
x1 = mineralogy_inc_pca_df.iloc[:,:1]
x2 = mineralogy_inc_pca_df.iloc[:,1:2]
x3 = mineralogy_inc_pca_df.iloc[:,2:3]
x4 = mineralogy_inc_pca_df.iloc[:,3:4]
x5 = mineralogy_inc_pca_df.iloc[:,4:5]
x6 = mineralogy_inc_pca_df.iloc[:,5:6]
x7 = mineralogy_inc_pca_df.iloc[:,6:7]

x9 = coordinates.iloc[:,:1]
x10 = coordinates.iloc[:,1:2]

In [75]:
scaler1 = StandardScaler().fit(x1)
scaler2 = StandardScaler().fit(x2)
scaler3 = StandardScaler().fit(x3)
scaler4 = StandardScaler().fit(x4)
scaler5 = StandardScaler().fit(x5)
scaler6 = StandardScaler().fit(x6)
scaler7 = StandardScaler().fit(x7)
scaler8 = StandardScaler().fit(x8)
scaler9 = StandardScaler().fit(x9)
scaler10 = StandardScaler().fit(x10)

In [76]:
x1_scaled = scaler1.transform(x1)
x2_scaled = scaler2.transform(x2)
x3_scaled = scaler3.transform(x3)
x4_scaled = scaler4.transform(x4)
x5_scaled = scaler5.transform(x5)
x6_scaled = scaler6.transform(x6)
x7_scaled = scaler7.transform(x7)
x8_scaled = scaler8.transform(x8)
x9_scaled = scaler9.transform(x9)
x10_scaled = scaler10.transform(x10)

In [77]:
scaler9.mean_

array([7606529.46350275])

In [78]:
scaled_inc_pca_df = mineralogy_inc_pca_df.iloc[:,:0]

In [79]:
x1_df = pd.DataFrame(data =x1_scaled,columns=['PC01'])
x2_df = pd.DataFrame(data =x2_scaled,columns=['PC02'])
x3_df = pd.DataFrame(data =x3_scaled,columns=['PC03'])
x4_df = pd.DataFrame(data =x4_scaled,columns=['PC04'])
x5_df = pd.DataFrame(data =x5_scaled,columns=['PC05'])
x6_df = pd.DataFrame(data =x6_scaled,columns=['PC06'])
x7_df = pd.DataFrame(data =x7_scaled,columns=['PC07'])
x8_df = pd.DataFrame(data =x8_scaled,columns=['time'])
x9_df = pd.DataFrame(data =x9_scaled,columns=['X_UTM'])
x10_df = pd.DataFrame(data =x10_scaled,columns=['Y_UTM'])

In [80]:
scaled_inc_pca_df = pd.concat([mineralogy_inc_pca_df.iloc[:,:0], x1_df, x2_df, x3_df, x4_df, x5_df, x6_df, x7_df, x8_df, x9_df, x10_df], axis=1, sort=True)
scaled_inc_pca_df.index = np.arange(1, len(scaled_inc_pca_df) + 1)
scaled_inc_pca_df = scaled_inc_pca_df.dropna()

scaled_time_pca_df = pd.concat([mineralogy_inc_pca_df.iloc[:,:0], x1_df, x2_df, x3_df, x4_df, x5_df, x6_df, x7_df, x8_df], axis=1, sort=True)
scaled_time_pca_df.index = np.arange(1, len(scaled_time_pca_df) + 1)
scaled_time_pca_df = scaled_time_pca_df.dropna()

scaled_coordinates_pca_df = pd.concat([mineralogy_inc_pca_df.iloc[:,:0], x1_df, x2_df, x3_df, x4_df, x5_df, x6_df, x7_df, x9_df, x10_df], axis=1, sort=True)
scaled_coordinates_pca_df.index = np.arange(1, len(scaled_coordinates_pca_df) + 1)
scaled_coordinates_pca_df = scaled_coordinates_pca_df.dropna()

In [81]:
scaled_coordinates_pca_df

,PC01,PC02,PC03,PC04,PC05,PC06,PC07,X_UTM,Y_UTM
1,-0.821697,0.073729,-1.747331,1.849427,0.424171,-2.688559,-0.670874,-0.184146,0.182484
2,-0.971980,1.049750,-0.742744,0.614839,1.802281,1.397826,-0.565449,-0.219297,0.196792
3,-1.620936,1.255681,0.204918,-0.116210,1.696146,-2.627857,-3.105933,-0.163724,0.208783
4,-1.050704,-0.770527,1.285680,3.028266,0.627044,0.068766,-0.497492,-0.176159,0.205965
5,-1.163557,0.104422,1.082868,-0.397029,-3.296363,-2.878734,-1.418440,-0.161108,0.218651
...,...,...,...,...,...,...,...,...,...
4655,1.635276,-1.337046,1.190968,1.710257,-0.671667,-1.057086,0.206893,-0.157985,0.215949
4656,2.296487,-0.787293,0.100543,-0.102332,-0.322927,-1.786802,0.118447,-0.187642,0.185681
4657,2.167530,-1.402840,0.512226,0.110387,-0.002831,-0.375974,-0.767273,-0.186847,0.200860
4658,2.005894,-0.910735,0.281555,-0.393594,-1.399058,-0.052875,2.234582,-0.173634,0.205163


In [82]:
scaled_time_pca_df

,PC01,PC02,PC03,PC04,PC05,PC06,PC07,time
1,-0.821697,0.073729,-1.747331,1.849427,0.424171,-2.688559,-0.670874,0.467685
2,-0.971980,1.049750,-0.742744,0.614839,1.802281,1.397826,-0.565449,-2.133358
3,-1.620936,1.255681,0.204918,-0.116210,1.696146,-2.627857,-3.105933,0.467685
4,-1.050704,-0.770527,1.285680,3.028266,0.627044,0.068766,-0.497492,0.467685
5,-1.163557,0.104422,1.082868,-0.397029,-3.296363,-2.878734,-1.418440,0.467685
...,...,...,...,...,...,...,...,...
4655,1.635276,-1.337046,1.190968,1.710257,-0.671667,-1.057086,0.206893,0.467685
4656,2.296487,-0.787293,0.100543,-0.102332,-0.322927,-1.786802,0.118447,0.467685
4657,2.167530,-1.402840,0.512226,0.110387,-0.002831,-0.375974,-0.767273,-0.399329
4658,2.005894,-0.910735,0.281555,-0.393594,-1.399058,-0.052875,2.234582,0.467685


In [83]:
preproc.save_obj(scaled_inc_pca_df, "scaled_inc_pca_df")
preproc.save_obj(scaled_time_pca_df, "scaled_time_pca_df")
preproc.save_obj(scaled_coordinates_pca_df, "scaled_coordinates_pca_df")

-----

# grid points cluster analysis

## Area 1 

In [5]:
area1_int = pd.read_excel("../_CIPW/CIPW/AREA1/interpolated_data.xlsx", index_col=0)
area1_int = area1_int.dropna()
area1_int.drop(["oth", ], axis = 1, inplace = True)
area1_int = preproc.replace_zero(area1_int, 0.01)


In [6]:
area1_int.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55413 entries, 2809 to 205544
Data columns (total 10 columns):
SiO2     55413 non-null float64
TiO2     55413 non-null float64
Al2O3    55413 non-null float64
Fe2O3    55413 non-null float64
FeO      55413 non-null float64
MnO      55413 non-null float64
MgO      55413 non-null float64
CaO      55413 non-null float64
Na2O     55413 non-null float64
K2O      55413 non-null float64
dtypes: float64(10)
memory usage: 4.7 MB


In [7]:
area1_int

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O
2809,70.222982,0.306007,15.037751,0.652161,2.100375,0.039497,1.063120,2.105502,3.870846,3.740172
2810,70.240242,0.305221,15.034542,0.652185,2.095248,0.039421,1.059408,2.098104,3.870482,3.742184
2811,70.253542,0.304611,15.032047,0.652144,2.091362,0.039362,1.056553,2.092448,3.870193,3.743731
2812,70.241073,0.312743,15.003450,0.619717,2.223126,0.042901,1.036778,2.105143,3.909642,3.782753
2813,70.243463,0.312634,15.002904,0.619474,2.222875,0.042899,1.036236,2.104256,3.909656,3.783102
...,...,...,...,...,...,...,...,...,...,...
205097,71.839678,0.283426,14.517509,0.948295,1.714704,0.045674,0.624198,1.249518,4.051043,4.229271
205541,71.822233,0.284638,14.521715,0.958462,1.712651,0.045724,0.626124,1.251051,4.052232,4.228004
205542,71.824211,0.284493,14.521284,0.957263,1.712859,0.045716,0.625929,1.250916,4.052057,4.228092
205543,71.826556,0.284328,14.520734,0.955941,1.713074,0.045707,0.625675,1.250699,4.051882,4.228246


In [8]:
area1_int_clr = preproc.clr(area1_int)
area1_int_clr.head()

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O
2809,3.617717,-1.818107,2.076606,-1.061423,0.108158,-3.865485,-0.572750,0.110596,0.719515,0.685173
2810,3.619309,-1.819333,2.077737,-1.060040,0.107059,-3.866074,-0.574903,0.108421,0.720766,0.687057
2811,3.620542,-1.820289,2.078616,-1.059059,0.106247,-3.866515,-0.576557,0.106766,0.721736,0.688514
2812,3.607553,-1.806754,2.063899,-1.122872,0.154534,-3.793236,-0.608263,0.100003,0.719065,0.686071
2813,3.607763,-1.806927,2.064040,-1.123088,0.154597,-3.793120,-0.608609,0.099758,0.719245,0.686340


In [9]:
area1_int_pca = preproc.pca(area1_int_clr)
preproc.pca_variance(area1_int_pca)

2 PCA components  out of 10 components with variance sum 0.9773112140707415 needed for obtaining sum of variance > 0.95


array([6.52307621e-01, 3.25003593e-01, 2.26887859e-02, 5.47025444e-31,
       1.72784347e-31, 1.36543201e-31, 8.31338518e-32, 6.65649746e-32,
       5.62173424e-32, 3.73045601e-32])

In [10]:
area1_int_df = preproc.create_pca_df(area1_int_pca, area1_int_clr)

In [11]:
area1_int_df

,PC01,PC02,PC03,PC04,PC05,PC06,PC07,PC08,PC09,PC10
2809,-0.871616,-0.061954,-0.178992,-3.667464e-16,-6.732535e-17,2.033403e-16,-1.077045e-16,-4.769786e-17,-2.249744e-16,-2.429622e-16
2810,-0.867083,-0.061057,-0.180370,-7.451425e-16,1.624210e-17,-1.705633e-16,-3.907189e-16,-2.133986e-16,-4.304799e-17,-2.793085e-16
2811,-0.863637,-0.060284,-0.181416,-9.220021e-16,6.328708e-18,-1.430243e-17,9.122456e-17,8.590459e-17,1.582475e-16,1.372954e-16
2812,-0.881656,-0.001032,-0.083585,-7.859609e-16,-2.094071e-16,-7.524761e-16,-1.179243e-16,1.622333e-16,2.295189e-16,-1.097492e-16
2813,-0.881230,-0.000526,-0.083510,-7.583104e-16,4.920769e-17,-2.476059e-16,-4.686063e-17,-2.128640e-16,5.045648e-16,-1.338189e-17
...,...,...,...,...,...,...,...,...,...,...
205097,-0.049269,-0.211667,0.023654,-3.985039e-16,1.049181e-16,-3.073668e-17,-2.316495e-16,8.972486e-17,4.643697e-16,-6.205094e-17
205541,-0.047306,-0.221904,0.022992,-5.638857e-16,1.370082e-18,-1.317205e-16,3.132650e-17,-9.524689e-17,-4.777928e-16,1.173006e-17
205542,-0.047587,-0.220724,0.022998,-6.280217e-16,4.503544e-16,-4.097656e-16,-8.273617e-17,2.521313e-16,-7.196328e-17,8.275516e-18
205543,-0.047827,-0.219399,0.023041,-7.122765e-16,2.530445e-17,1.490451e-16,-2.164953e-16,-1.068615e-16,-3.819137e-17,-1.552907e-17


In [12]:
preproc.save_obj(area1_int_df, "area1_int_df")

-----

## Area 2

In [13]:
area2_int = pd.read_excel("../_CIPW/CIPW/AREA2/interpolated_data.xlsx", index_col=0)
area2_int = area2_int.dropna()
area2_int.drop(["oth", ], axis = 1, inplace = True)
area2_int = preproc.replace_zero(area2_int, 0.01)

In [14]:
area2_int.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47039 entries, 3579 to 133145
Data columns (total 10 columns):
SiO2     47039 non-null float64
TiO2     47039 non-null float64
Al2O3    47039 non-null float64
Fe2O3    47039 non-null float64
FeO      47039 non-null float64
MnO      47039 non-null float64
MgO      47039 non-null float64
CaO      47039 non-null float64
Na2O     47039 non-null float64
K2O      47039 non-null float64
dtypes: float64(10)
memory usage: 3.9 MB


In [15]:
area2_int

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O
3579,70.566092,0.345193,15.086986,1.092977,2.186120,0.072841,0.867590,1.863684,3.499057,3.667570
3580,70.564361,0.345266,15.087666,1.091705,2.187278,0.072846,0.868059,1.864769,3.498852,3.667177
3581,70.563901,0.345274,15.088025,1.090232,2.188203,0.072844,0.868304,1.865475,3.498670,3.666974
3582,70.564769,0.345214,15.088038,1.088623,2.188843,0.072834,0.868308,1.865758,3.498518,3.666974
3826,68.678145,0.439813,15.523218,1.071121,2.704864,0.082557,1.265489,2.584797,3.427112,3.389030
...,...,...,...,...,...,...,...,...,...,...
133140,67.026320,0.531393,15.776172,1.280568,3.009275,0.091436,1.641670,3.174217,3.384517,3.194583
133141,67.026397,0.531418,15.776104,1.281622,3.008660,0.091441,1.641538,3.173739,3.384620,3.194643
133142,67.025330,0.531514,15.776170,1.283051,3.008122,0.091452,1.641658,3.173585,3.384713,3.194588
133144,67.020400,0.531871,15.776645,1.286538,3.007398,0.091491,1.642543,3.174191,3.384847,3.194180


In [16]:
area2_int_clr = preproc.clr(area2_int)
area2_int_clr.head()

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O
3579,3.537468,-1.782734,1.994750,-0.630177,0.063046,-3.338565,-0.861118,-0.096527,0.533411,0.580447
3580,3.537380,-1.782586,1.994732,-0.631404,0.063513,-3.338552,-0.860641,-0.096008,0.533290,0.580277
3581,3.537410,-1.782527,1.994792,-0.632718,0.063972,-3.338545,-0.860322,-0.095593,0.533274,0.580258
3582,3.537559,-1.782563,1.994930,-0.634059,0.064401,-3.338544,-0.860181,-0.095305,0.533367,0.580395
3826,3.393708,-1.657128,1.906614,-0.767017,0.159329,-3.329984,-0.600264,0.113925,0.395995,0.384821


In [17]:
area2_int_pca = preproc.pca(area2_int_clr)
preproc.pca_variance(area2_int_pca)

2 PCA components  out of 10 components with variance sum 1.0 needed for obtaining sum of variance > 0.95


array([6.43032079e-01, 3.56967921e-01, 3.41191414e-31, 1.49326190e-31,
       9.56613785e-32, 8.06639250e-32, 5.75176005e-32, 5.03115808e-32,
       4.47368603e-32, 3.10087590e-32])

In [18]:
area2_int_df = preproc.create_pca_df(area2_int_pca, area2_int_clr)

In [19]:
area2_int_df

,PC01,PC02,PC03,PC04,PC05,PC06,PC07,PC08,PC09,PC10
3579,0.034400,-0.215940,-1.834814e-16,-4.199368e-16,-6.281014e-17,-2.118512e-16,8.866309e-17,5.129524e-17,8.948691e-17,2.454378e-16
3580,0.033310,-0.214889,2.707808e-16,6.026089e-16,5.752368e-17,1.506464e-16,-6.616786e-17,-1.955281e-16,-1.721296e-16,-1.718674e-17
3581,0.032483,-0.213649,-1.388944e-17,-1.660980e-16,-4.492476e-17,4.838428e-16,1.285298e-16,2.963508e-16,-7.976224e-18,4.383250e-17
3582,0.031967,-0.212275,-3.201356e-17,5.873404e-17,-2.684214e-16,2.627717e-16,3.560237e-17,-1.123423e-16,1.218733e-16,1.013452e-16
3826,-0.457255,-0.222506,3.489372e-16,-6.304948e-16,3.212107e-16,2.619495e-18,-1.632224e-17,-9.687838e-17,1.106057e-16,-2.149205e-19
...,...,...,...,...,...,...,...,...,...,...
133140,-0.733272,-0.409852,8.289992e-16,-2.844263e-16,-1.499328e-16,-1.515378e-18,-1.529536e-16,-6.333319e-17,1.007722e-16,-2.197670e-16
133141,-0.732883,-0.410611,5.841354e-16,-2.964609e-16,3.344490e-16,2.212887e-16,-2.545138e-17,-1.087980e-16,-7.684588e-17,2.224536e-16
133142,-0.732593,-0.411653,6.328189e-16,-3.391264e-16,-3.480494e-16,2.856421e-17,-7.462779e-17,-7.417739e-17,-1.050051e-16,9.262674e-17
133144,-0.732359,-0.414225,2.689790e-16,-1.977553e-16,4.085971e-17,-5.774086e-16,1.383334e-16,1.367974e-16,-1.338760e-16,-1.223390e-17


In [20]:
preproc.save_obj(area2_int_df, "area2_int_df")

----

## Area 3

In [21]:
area3_int = pd.read_excel("../_CIPW/CIPW/AREA3/interpolated_data.xlsx", index_col=0)
area3_int = area3_int.dropna()
area3_int.drop(["oth", ], axis = 1, inplace = True)
area3_int = preproc.replace_zero(area3_int, 0.01)

In [22]:
area3_int.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54455 entries, 11189 to 160544
Data columns (total 10 columns):
SiO2     54455 non-null float64
TiO2     54455 non-null float64
Al2O3    54455 non-null float64
Fe2O3    54455 non-null float64
FeO      54455 non-null float64
MnO      54455 non-null float64
MgO      54455 non-null float64
CaO      54455 non-null float64
Na2O     54455 non-null float64
K2O      54455 non-null float64
dtypes: float64(10)
memory usage: 4.6 MB


In [23]:
area3_int

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O
11189,68.973826,0.384729,15.490616,2.105048,2.125391,0.063841,1.003265,2.264021,3.407279,3.337133
11190,68.933533,0.386681,15.498923,2.110996,2.133132,0.064013,1.010837,2.276947,3.406195,3.331904
11191,68.893888,0.388613,15.507131,2.116467,2.140859,0.064183,1.018360,2.289786,3.405126,3.326788
11192,68.856651,0.390440,15.514863,2.121303,2.148205,0.064343,1.025487,2.301946,3.404120,3.322008
11596,69.065490,0.380060,15.469415,2.099817,2.105257,0.063444,0.985195,2.232583,3.409713,3.348840
...,...,...,...,...,...,...,...,...,...,...
160540,65.844594,0.575363,16.199450,1.031606,3.315901,0.073653,1.849313,3.758044,3.304477,3.066352
160541,65.888449,0.572938,16.194183,1.038694,3.300992,0.073544,1.837354,3.738182,3.306089,3.069814
160542,65.963747,0.568772,16.185019,1.050153,3.275856,0.073351,1.816909,3.704308,3.308833,3.075885
160543,65.954892,0.569303,16.186553,1.042429,3.282323,0.073342,1.819655,3.709889,3.308405,3.075937


In [24]:
area3_int_clr = preproc.clr(area3_int)
area3_int_clr.head()

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O
11189,3.432017,-1.756925,1.938524,-0.057372,-0.047754,-3.553076,-0.798450,0.015432,0.424204,0.403402
11190,3.428883,-1.754417,1.936510,-0.057100,-0.046669,-3.552921,-0.793481,0.018575,0.421336,0.399284
11191,3.425809,-1.751929,1.934542,-0.057010,-0.045551,-3.552769,-0.788565,0.021700,0.418524,0.395249
11192,3.422931,-1.749577,1.932703,-0.057065,-0.044463,-3.552624,-0.783928,0.024659,0.415891,0.391474
11596,3.439188,-1.763293,1.942997,-0.054017,-0.051430,-3.553460,-0.810784,0.007292,0.430760,0.412747


In [25]:
area3_int_pca = preproc.pca(area3_int_clr)
preproc.pca_variance(area3_int_pca)

2 PCA components  out of 10 components with variance sum 1.0 needed for obtaining sum of variance > 0.95


array([5.98138421e-01, 4.01861579e-01, 3.87080825e-31, 1.31333072e-31,
       1.10359749e-31, 9.12883257e-32, 6.18384818e-32, 5.37724228e-32,
       4.64576740e-32, 3.50855223e-32])

In [26]:
area3_int_df = preproc.create_pca_df(area3_int_pca, area3_int_clr)

In [27]:
area3_int_df

,PC01,PC02,PC03,PC04,PC05,PC06,PC07,PC08,PC09,PC10
11189,0.378920,-0.911470,5.439131e-16,-1.952341e-16,-2.814163e-16,2.566793e-16,-4.017168e-16,-2.583177e-16,-1.439430e-16,-5.706962e-17
11190,0.372451,-0.917743,1.876171e-16,8.945326e-17,2.833851e-16,-1.164986e-16,1.366745e-16,-3.112835e-16,-1.473322e-16,-1.207413e-16
11191,0.365942,-0.923789,3.212005e-16,-3.298020e-16,-1.041261e-16,-2.887258e-16,-8.421969e-17,-1.789867e-16,-5.016274e-17,1.234255e-17
11192,0.359717,-0.929365,1.312915e-16,-2.854636e-16,1.055485e-16,2.685475e-17,-1.264535e-16,4.191175e-17,-1.948273e-16,-2.054843e-16
11596,0.397430,-0.899513,5.705767e-16,1.050845e-16,-8.080592e-17,7.356383e-18,-2.336581e-17,1.270848e-16,-7.990720e-17,-3.184433e-16
...,...,...,...,...,...,...,...,...,...,...
160540,-0.778978,-0.744624,1.247884e-16,-4.458693e-17,-5.274837e-16,4.295536e-16,-7.294304e-18,-2.613299e-16,-2.176553e-16,-2.134437e-17
160541,-0.767105,-0.745673,6.640944e-16,1.317419e-16,6.051890e-17,-7.647030e-17,-5.084030e-16,7.613067e-17,-2.469211e-16,-4.755967e-17
160542,-0.747192,-0.746888,1.307851e-17,-4.474222e-16,3.963108e-17,-1.378375e-17,-4.195711e-16,-1.828671e-16,-2.264948e-16,2.980998e-17
160543,-0.753829,-0.742470,6.015401e-16,-4.235754e-16,-4.340958e-18,4.227186e-16,-2.789172e-17,1.825506e-16,-1.102268e-16,-6.836111e-17


In [28]:
preproc.save_obj(area3_int_df, "area3_int_df")

----

## Area 4

In [29]:
area4_int = pd.read_excel("../_CIPW/CIPW/AREA4/interpolated_data.xlsx", index_col=0)
area4_int = area4_int.dropna()
area4_int.drop(["oth", ], axis = 1, inplace = True)
area4_int = preproc.replace_zero(area4_int, 0.01)

In [30]:
area4_int.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59479 entries, 2499 to 134976
Data columns (total 10 columns):
SiO2     59479 non-null float64
TiO2     59479 non-null float64
Al2O3    59479 non-null float64
Fe2O3    59479 non-null float64
FeO      59479 non-null float64
MnO      59479 non-null float64
MgO      59479 non-null float64
CaO      59479 non-null float64
Na2O     59479 non-null float64
K2O      59479 non-null float64
dtypes: float64(10)
memory usage: 5.0 MB


In [31]:
area4_int

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O
2499,68.157353,0.389587,15.940766,1.459134,2.561572,0.056441,1.543992,3.051329,3.567657,2.134805
2500,68.150893,0.389888,15.941639,1.460142,2.562972,0.056480,1.545360,3.053444,3.567597,2.134063
2501,68.146669,0.390094,15.942191,1.460837,2.563919,0.056509,1.546200,3.054770,3.567559,2.133670
2502,68.144610,0.390206,15.942435,1.461226,2.564424,0.056529,1.546533,3.055337,3.567544,2.133604
2503,68.144608,0.390229,15.942387,1.461325,2.564508,0.056540,1.546389,3.055187,3.567550,2.133841
...,...,...,...,...,...,...,...,...,...,...
134972,69.341569,0.321180,15.276715,0.844648,1.992609,0.033076,0.895321,1.911588,3.143461,5.509776
134973,69.349371,0.320701,15.273379,0.844562,1.989720,0.033033,0.893315,1.908378,3.143485,5.514530
134974,69.359296,0.320101,15.269328,0.844390,1.986195,0.032981,0.890875,1.904469,3.143549,5.519897
134975,69.371339,0.319381,15.264571,0.844127,1.982046,0.032919,0.888009,1.899876,3.143655,5.525843


In [32]:
area4_int_clr = preproc.clr(area4_int)
area4_int_clr.head()

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O
2499,3.414602,-1.749886,1.961663,-0.429374,0.133404,-3.681773,-0.372846,0.308360,0.464692,-0.048842
2500,3.414120,-1.749499,1.961330,-0.429071,0.133563,-3.681472,-0.372348,0.308666,0.464288,-0.049577
2501,3.413793,-1.749236,1.961100,-0.428860,0.133668,-3.681217,-0.372069,0.308835,0.464013,-0.050026
2502,3.413618,-1.749094,1.960970,-0.428738,0.133720,-3.681012,-0.371999,0.308875,0.463863,-0.050202
2503,3.413586,-1.749067,1.960936,-0.428702,0.133721,-3.680857,-0.372123,0.308795,0.463833,-0.050122


In [33]:
area4_int_pca = preproc.pca(area4_int_clr)
preproc.pca_variance(area4_int_pca)

4 PCA components  out of 10 components with variance sum 0.9999999999999999 needed for obtaining sum of variance > 0.95


array([6.44692309e-01, 1.85364036e-01, 1.06554614e-01, 6.33890403e-02,
       1.86013230e-31, 9.80462890e-32, 7.54045993e-32, 5.44979498e-32,
       4.32215351e-32, 3.60683654e-32])

In [34]:
area4_int_df = preproc.create_pca_df(area4_int_pca, area4_int_clr)

In [35]:
area4_int_df

,PC01,PC02,PC03,PC04,PC05,PC06,PC07,PC08,PC09,PC10
2499,0.485689,-0.125740,-0.055342,0.246923,-1.039289e-16,-1.302962e-16,-2.587433e-16,-7.067557e-17,-2.311174e-16,1.870939e-16
2500,0.486885,-0.126015,-0.055743,0.246641,-3.899310e-16,-5.753650e-16,-3.178221e-16,1.094210e-17,-1.028628e-16,-6.561663e-17
2501,0.487637,-0.126147,-0.056041,0.246396,1.749958e-17,5.290171e-16,-3.432184e-16,-4.959165e-16,2.595399e-16,1.489980e-16
2502,0.487965,-0.126145,-0.056236,0.246190,-3.953032e-16,1.114381e-16,-4.375435e-16,1.184405e-16,1.459494e-16,9.663860e-17
2503,0.487892,-0.126016,-0.056330,0.246027,-2.536859e-16,3.304752e-16,-4.077117e-16,5.206356e-16,-4.974291e-19,2.463790e-16
...,...,...,...,...,...,...,...,...,...,...
134972,-0.823145,-0.212690,-0.299398,0.431477,-7.193210e-16,5.551460e-16,8.122340e-17,-9.539794e-17,2.986868e-16,-1.868084e-16
134973,-0.825401,-0.210903,-0.299287,0.432355,-3.077927e-16,4.892259e-16,4.208353e-16,-3.757978e-17,1.507400e-16,7.176897e-17
134974,-0.828128,-0.208760,-0.299055,0.433406,-3.048143e-16,1.673033e-16,3.445884e-16,5.202831e-17,9.141722e-17,9.053739e-17
134975,-0.831322,-0.206267,-0.298698,0.434626,-1.881016e-17,3.270835e-16,1.704870e-16,1.612700e-16,-2.190454e-16,-3.638746e-17


In [36]:
preproc.save_obj(area4_int_df, "area4_int_df")

## Area 5

In [37]:
area5_int = pd.read_excel("../_CIPW/CIPW/AREA5/interpolated_data.xlsx", index_col=0)
area5_int = area5_int.dropna()
area5_int.drop(["oth", ], axis = 1, inplace = True)
area5_int = preproc.replace_zero(area5_int, 0.01)

In [38]:
area5_int.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60209 entries, 1996 to 123586
Data columns (total 10 columns):
SiO2     60209 non-null float64
TiO2     60209 non-null float64
Al2O3    60209 non-null float64
Fe2O3    60209 non-null float64
FeO      60209 non-null float64
MnO      60209 non-null float64
MgO      60209 non-null float64
CaO      60209 non-null float64
Na2O     60209 non-null float64
K2O      60209 non-null float64
dtypes: float64(10)
memory usage: 5.1 MB


In [39]:
area5_int

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O
1996,64.492324,0.779771,16.021955,1.304960,3.469338,0.117915,2.322952,3.577475,3.188924,3.516040
1997,64.491820,0.779800,16.021959,1.305092,3.469338,0.117909,2.323196,3.577614,3.188927,3.515996
1998,64.491327,0.779828,16.021964,1.305219,3.469341,0.117902,2.323433,3.577751,3.188929,3.515952
1999,64.490847,0.779855,16.021969,1.305340,3.469347,0.117896,2.323662,3.577884,3.188930,3.515910
2000,64.490387,0.779882,16.021974,1.305454,3.469356,0.117891,2.323880,3.578012,3.188931,3.515870
...,...,...,...,...,...,...,...,...,...,...
123582,70.138886,0.345292,15.337506,0.637853,2.070939,0.049323,0.858240,2.102874,3.323972,4.203421
123583,70.138775,0.345299,15.337522,0.637823,2.071004,0.049323,0.858259,2.102911,3.323960,4.203422
123584,70.138706,0.345304,15.337530,0.637794,2.071055,0.049324,0.858270,2.102936,3.323949,4.203427
123585,70.138680,0.345306,15.337530,0.637765,2.071092,0.049324,0.858271,2.102947,3.323940,4.203435


In [40]:
area5_int_clr = preproc.clr(area5_int)
area5_int_clr.head()

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O
1996,3.106685,-1.308616,1.714099,-0.793689,0.184102,-3.197650,-0.217023,0.214796,0.099822,0.197474
1997,3.106657,-1.308600,1.714078,-0.793608,0.184082,-3.197728,-0.216938,0.214814,0.099803,0.197441
1998,3.106629,-1.308584,1.714059,-0.793531,0.184063,-3.197802,-0.216856,0.214833,0.099783,0.197408
1999,3.106602,-1.308569,1.714039,-0.793458,0.184045,-3.197872,-0.216777,0.214850,0.099764,0.197377
2000,3.106576,-1.308553,1.714021,-0.793390,0.184028,-3.197937,-0.216703,0.214867,0.099745,0.197346


In [41]:
area5_int_pca = preproc.pca(area5_int_clr)
preproc.pca_variance(area5_int_pca)

3 PCA components  out of 10 components with variance sum 1.0 needed for obtaining sum of variance > 0.95


array([6.09225681e-01, 2.32476806e-01, 1.58297513e-01, 3.98711542e-31,
       1.21647309e-31, 9.40885894e-32, 6.13241177e-32, 6.03063262e-32,
       5.14894195e-32, 3.09583294e-32])

In [42]:
area5_int_df = preproc.create_pca_df(area5_int_pca, area5_int_clr)

In [43]:
area5_int_df

,PC01,PC02,PC03,PC04,PC05,PC06,PC07,PC08,PC09,PC10
1996,-1.404360,-0.816303,0.365033,-4.081481e-16,3.902347e-16,-3.556127e-16,-2.838719e-17,1.450981e-16,-1.870483e-16,2.118815e-16
1997,-1.404402,-0.816395,0.364919,-6.622449e-16,-3.618581e-16,-6.747147e-16,-3.307569e-16,-4.321083e-17,1.870591e-16,-1.552170e-16
1998,-1.404444,-0.816486,0.364810,-8.289077e-16,-4.977368e-16,-7.514300e-17,4.930142e-16,3.109769e-16,3.922534e-17,9.156759e-17
1999,-1.404485,-0.816572,0.364707,-9.128211e-16,-5.519901e-17,-5.802901e-16,-3.276596e-18,3.603118e-16,-8.580826e-17,-1.815730e-16
2000,-1.404526,-0.816654,0.364611,-7.009625e-16,-1.680333e-16,7.696369e-18,5.069393e-16,8.697547e-17,-1.880096e-16,1.298028e-16
...,...,...,...,...,...,...,...,...,...,...
123582,-0.516033,0.133608,0.090183,-4.516425e-16,3.252055e-16,2.876175e-16,1.159547e-17,7.687785e-17,2.122814e-17,2.003950e-16
123583,-0.516086,0.133643,0.090197,-3.452497e-16,-5.701888e-16,4.549955e-16,2.243574e-16,1.569767e-16,1.876207e-17,-1.915397e-16
123584,-0.516128,0.133679,0.090211,-8.443746e-16,-7.192301e-18,2.692906e-16,2.349037e-16,2.786820e-16,1.400359e-16,-9.615518e-18
123585,-0.516157,0.133716,0.090226,-2.673639e-16,2.654459e-16,-1.868731e-16,-6.503191e-18,3.085155e-16,2.430331e-17,-1.814433e-16


In [44]:
preproc.save_obj(area5_int_df, "area5_int_df")

---

# Zoomed in areas

## Area 2a

In [45]:
area2a_int = pd.read_excel("../_CIPW/CIPW/AREA2a/interpolated_data.xlsx", index_col=0)
area2a_int = area2a_int.dropna()
area2a_int.drop(["oth", ], axis = 1, inplace = True)
area2a_int = preproc.replace_zero(area2a_int, 0.01)

In [46]:
area2a_int.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49861 entries, 7431 to 126622
Data columns (total 10 columns):
SiO2     49861 non-null float64
TiO2     49861 non-null float64
Al2O3    49861 non-null float64
Fe2O3    49861 non-null float64
FeO      49861 non-null float64
MnO      49861 non-null float64
MgO      49861 non-null float64
CaO      49861 non-null float64
Na2O     49861 non-null float64
K2O      49861 non-null float64
dtypes: float64(10)
memory usage: 4.2 MB


In [47]:
area2a_int

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O
7431,70.169757,0.364318,15.208769,0.815537,2.535378,0.057258,0.602557,1.777622,3.546316,4.164508
7432,70.136994,0.365755,15.218807,0.820647,2.541740,0.057403,0.606876,1.787653,3.546100,4.158249
7433,70.102085,0.367273,15.229450,0.825746,2.548705,0.057561,0.611545,1.798436,3.545863,4.151606
7434,70.065027,0.368874,15.240687,0.830820,2.556279,0.057730,0.616571,1.809980,3.545601,4.144584
7435,70.025824,0.370556,15.252509,0.835860,2.564466,0.057911,0.621958,1.822293,3.545314,4.137189
...,...,...,...,...,...,...,...,...,...,...
126270,70.780122,0.315798,14.908608,1.592934,2.015031,0.064971,0.549682,1.325033,3.622341,4.084271
126271,70.741669,0.317464,14.921511,1.598063,2.023298,0.065096,0.554245,1.335613,3.621693,4.078165
126272,70.703570,0.319119,14.934199,1.603241,2.031456,0.065217,0.558757,1.346127,3.621035,4.072166
126273,70.665932,0.320760,14.946640,1.608452,2.039482,0.065334,0.563205,1.356543,3.620369,4.066288


In [48]:
area2a_int_clr = preproc.clr(area2a_int)
area2a_int_clr.head()

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O
7431,3.591866,-1.668779,2.062821,-0.862960,0.271291,-3.519245,-1.165624,-0.083775,0.606858,0.767547
7432,3.588736,-1.667507,2.060818,-0.859376,0.271135,-3.519367,-1.161145,-0.080810,0.604135,0.763380
7433,3.585436,-1.666166,2.058714,-0.855985,0.271069,-3.519430,-1.156283,-0.077599,0.601265,0.758979
7434,3.581967,-1.664758,2.056512,-0.852799,0.271096,-3.519433,-1.151040,-0.074141,0.598251,0.754345
7435,3.578332,-1.663284,2.054211,-0.849827,0.271218,-3.519375,-1.145416,-0.070437,0.595094,0.749484


In [49]:
area2a_int_pca = preproc.pca(area2a_int_clr)
preproc.pca_variance(area2a_int_pca)

3 PCA components  out of 10 components with variance sum 1.0 needed for obtaining sum of variance > 0.95


array([7.29495762e-01, 1.96899961e-01, 7.36042775e-02, 1.20748029e-30,
       1.75799164e-31, 1.65797557e-31, 1.20431887e-31, 1.05085204e-31,
       7.09153223e-32, 5.03585958e-32])

In [50]:
area2a_int_df = preproc.create_pca_df(area2a_int_pca, area2a_int_clr)

In [51]:
area2a_int_df

,PC01,PC02,PC03,PC04,PC05,PC06,PC07,PC08,PC09,PC10
7431,-0.008096,0.385552,-0.048256,-7.877876e-16,3.258096e-18,1.816809e-16,-1.631695e-16,-1.916457e-16,1.312121e-16,1.074816e-16
7432,-0.015158,0.380568,-0.050942,-1.178708e-15,2.525962e-16,-1.445052e-16,-1.403845e-17,-2.579701e-16,2.738796e-16,-5.707783e-18
7433,-0.022820,0.375572,-0.053497,-3.164760e-16,-8.622545e-17,-6.176908e-16,-7.559643e-17,-2.677997e-17,6.147118e-17,-1.234473e-16
7434,-0.031080,0.370576,-0.055911,5.926863e-17,-3.817237e-17,-4.059252e-16,-1.739658e-17,5.226838e-17,-2.419972e-16,-1.850179e-16
7435,-0.039937,0.365588,-0.058179,-5.674224e-16,-1.291990e-16,-3.305764e-16,8.282038e-17,3.489020e-16,-8.814998e-17,1.577516e-16
...,...,...,...,...,...,...,...,...,...,...
126270,0.391564,-0.274296,-0.243824,-1.939091e-16,-1.133038e-16,3.640794e-17,-3.512496e-16,-2.442475e-16,6.526597e-17,4.227203e-17
126271,0.381569,-0.275976,-0.245940,-1.962836e-17,1.634176e-16,-1.172850e-16,2.213132e-16,-3.814906e-16,2.205772e-16,-5.241572e-17
126272,0.371750,-0.277653,-0.248107,-6.217471e-16,-2.493144e-16,-4.744138e-17,1.231999e-16,-5.677189e-17,-1.179525e-16,2.774538e-16
126273,0.362130,-0.279324,-0.250317,-6.841537e-16,-4.932746e-17,-3.170661e-16,-2.846246e-17,2.663347e-16,2.586743e-17,-2.307437e-17


In [52]:
preproc.save_obj(area2a_int_df, "area2a_int_df")

## Area 2b

In [53]:
area2b_int = pd.read_excel("../_CIPW/CIPW/AREA2b/interpolated_data.xlsx", index_col=0)
area2b_int = area2b_int.dropna()
area2b_int.drop(["oth", ], axis = 1, inplace = True)
area2b_int = preproc.replace_zero(area2b_int, 0.01)

In [54]:
area2b_int.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66330 entries, 3448 to 117407
Data columns (total 10 columns):
SiO2     66330 non-null float64
TiO2     66330 non-null float64
Al2O3    66330 non-null float64
Fe2O3    66330 non-null float64
FeO      66330 non-null float64
MnO      66330 non-null float64
MgO      66330 non-null float64
CaO      66330 non-null float64
Na2O     66330 non-null float64
K2O      66330 non-null float64
dtypes: float64(10)
memory usage: 5.6 MB


In [55]:
area2b_int

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O
3448,67.803560,0.502334,15.400196,1.082592,3.103967,0.093106,1.982381,3.023204,3.154422,3.098104
3449,67.803553,0.502335,15.400130,1.082873,3.103788,0.093107,1.982578,3.023210,3.154403,3.097947
3450,67.391854,0.528271,15.435320,1.117539,3.182358,0.095171,2.135233,3.168716,3.139945,3.040987
3451,67.391762,0.528277,15.435255,1.117837,3.182187,0.095172,2.135472,3.168746,3.139923,3.040822
3452,67.391665,0.528283,15.435191,1.118133,3.182018,0.095174,2.135712,3.168779,3.139901,3.040657
...,...,...,...,...,...,...,...,...,...,...
117403,73.862425,0.162789,13.990487,0.578128,1.696639,0.055763,0.398726,1.052265,3.320295,4.330890
117404,73.862749,0.162778,13.990369,0.578172,1.696533,0.055762,0.398704,1.052199,3.320294,4.330878
117405,73.863025,0.162768,13.990265,0.578199,1.696446,0.055761,0.398682,1.052138,3.320294,4.330884
117406,73.863253,0.162759,13.990176,0.578209,1.696377,0.055759,0.398658,1.052084,3.320296,4.330908


In [56]:
area2b_int_clr = preproc.clr(area2b_int)
area2b_int_clr.head()

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O
3448,3.299541,-1.605565,1.817307,-0.837715,0.215607,-3.291091,-0.232775,0.189244,0.231731,0.213717
3449,3.299515,-1.605588,1.817277,-0.837481,0.215524,-3.291106,-0.232701,0.189220,0.231700,0.213640
3450,3.271125,-1.577545,1.797259,-0.828270,0.218223,-3.291483,-0.180824,0.213927,0.204806,0.172783
3451,3.271094,-1.577563,1.797225,-0.828033,0.218140,-3.291497,-0.180741,0.213907,0.204770,0.172699
3452,3.271063,-1.577581,1.797192,-0.827798,0.218057,-3.291512,-0.180658,0.213888,0.204733,0.172615


In [57]:
area2b_int_pca = preproc.pca(area2b_int_clr)
preproc.pca_variance(area2b_int_pca)

2 PCA components  out of 10 components with variance sum 1.0 needed for obtaining sum of variance > 0.95


array([8.00563670e-01, 1.99436330e-01, 8.32688926e-31, 3.05084146e-31,
       2.18110381e-31, 1.89122810e-31, 1.19301799e-31, 1.14375750e-31,
       9.78722058e-32, 6.46031328e-32])

In [58]:
area2b_int_df = preproc.create_pca_df(area2b_int_pca, area2b_int_clr)

In [59]:
area2b_int_df

,PC01,PC02,PC03,PC04,PC05,PC06,PC07,PC08,PC09,PC10
3448,1.309549,-0.681230,-3.246705e-16,-6.421615e-18,5.679719e-17,-1.622311e-16,-3.014871e-16,-1.361171e-16,1.195098e-16,-8.865511e-17
3449,1.309478,-0.681499,1.890755e-16,-4.363443e-17,-1.330152e-17,-2.427033e-16,-3.974586e-16,3.124443e-17,-1.946903e-17,2.543954e-16
3450,1.378581,-0.736312,4.353664e-16,-1.912506e-16,1.956898e-16,-5.685623e-16,2.376579e-16,-2.037367e-16,-3.990592e-17,1.675247e-17
3451,1.378522,-0.736591,1.240484e-16,3.682312e-17,1.682199e-16,-4.182570e-16,-2.599717e-16,-7.632520e-18,1.026395e-16,-8.432998e-17
3452,1.378465,-0.736868,-2.524243e-18,-8.223794e-16,2.694320e-16,-4.306436e-16,-2.300251e-16,1.618058e-16,-8.539968e-17,-1.104149e-16
...,...,...,...,...,...,...,...,...,...,...
117403,-0.202210,0.398705,4.620049e-16,-4.085090e-16,3.765421e-16,7.830187e-17,1.823795e-16,-2.674613e-16,-1.822003e-16,4.819953e-17
117404,-0.202332,0.398653,5.986516e-16,1.558270e-16,-5.349294e-16,-3.173112e-16,4.674978e-17,2.727189e-16,1.585589e-16,-2.108532e-16
117405,-0.202436,0.398628,1.084135e-15,-3.748825e-19,-2.362108e-16,3.805732e-16,-3.747573e-17,1.748645e-16,-5.604813e-17,6.468747e-17
117406,-0.202522,0.398628,-2.664524e-18,1.226521e-17,3.187803e-17,-5.485562e-17,-2.337335e-16,-2.677587e-17,2.962189e-16,-1.636137e-16


In [60]:
preproc.save_obj(area2b_int_df, "area2b_int_df")

## Area 3a

In [61]:
area3a_int = pd.read_excel("../_CIPW/CIPW/AREA3a/interpolated_data.xlsx", index_col=0)
area3a_int = area3a_int.dropna()
area3a_int.drop(["oth", ], axis = 1, inplace = True)
area3a_int = preproc.replace_zero(area3a_int, 0.01)

In [62]:
area3a_int.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53006 entries, 12193 to 147963
Data columns (total 10 columns):
SiO2     53006 non-null float64
TiO2     53006 non-null float64
Al2O3    53006 non-null float64
Fe2O3    53006 non-null float64
FeO      53006 non-null float64
MnO      53006 non-null float64
MgO      53006 non-null float64
CaO      53006 non-null float64
Na2O     53006 non-null float64
K2O      53006 non-null float64
dtypes: float64(10)
memory usage: 4.4 MB


In [63]:
area3a_int

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O
12193,72.015465,0.290437,14.495370,0.653807,1.400530,0.042271,0.623182,2.141107,3.352642,4.377819
12194,72.017364,0.290373,14.494655,0.653676,1.400141,0.042262,0.622907,2.140542,3.352723,4.378092
12195,72.019207,0.290311,14.493962,0.653548,1.399763,0.042253,0.622639,2.139993,3.352802,4.378356
12715,72.009125,0.290650,14.497747,0.654268,1.401824,0.042302,0.624110,2.142987,3.352371,4.376903
12716,72.011297,0.290577,14.496929,0.654120,1.401378,0.042291,0.623795,2.142340,3.352463,4.377215
...,...,...,...,...,...,...,...,...,...,...
147437,64.586536,0.539193,16.254790,1.161485,3.083182,0.078235,2.242151,4.524845,3.026354,3.529234
147438,64.584667,0.539254,16.255089,1.161418,3.083717,0.078247,2.242595,4.525563,3.026275,3.529077
147439,64.582764,0.539316,16.255394,1.161352,3.084261,0.078259,2.243050,4.526294,3.026195,3.528918
147440,64.580831,0.539379,16.255702,1.161289,3.084812,0.078271,2.243512,4.527035,3.026114,3.528756


In [64]:
area3a_int_clr = preproc.clr(area3a_int)
area3a_int_clr.head()

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O
12193,3.733150,-1.780100,2.130098,-0.968674,-0.206880,-3.707377,-1.016647,0.217592,0.666018,0.932820
12194,3.733332,-1.780163,2.130205,-0.968718,-0.207002,-3.707441,-1.016933,0.217484,0.666198,0.933038
12195,3.733509,-1.780225,2.130309,-0.968762,-0.207120,-3.707502,-1.017211,0.217379,0.666373,0.933250
12715,3.732538,-1.779889,2.129738,-0.968494,-0.206481,-3.707175,-1.015684,0.217946,0.665413,0.932087
12716,3.732746,-1.779962,2.129860,-0.968541,-0.206621,-3.707249,-1.016010,0.217822,0.665619,0.932336


In [65]:
area3a_int_pca = preproc.pca(area3a_int_clr)
preproc.pca_variance(area3a_int_pca)

2 PCA components  out of 10 components with variance sum 1.0 needed for obtaining sum of variance > 0.95


array([7.77008100e-01, 2.22991900e-01, 3.96146178e-31, 2.79529439e-31,
       2.14745572e-31, 1.61884712e-31, 1.06596548e-31, 6.20669951e-32,
       5.37122300e-32, 4.98712909e-32])

In [66]:
area3a_int_df = preproc.create_pca_df(area3a_int_pca, area3a_int_clr)

In [67]:
area3a_int_df

,PC01,PC02,PC03,PC04,PC05,PC06,PC07,PC08,PC09,PC10
12193,-0.728068,-0.519047,-6.432864e-16,-5.558494e-16,4.333730e-16,1.390537e-16,-1.012266e-16,-5.628536e-17,1.648310e-16,2.783748e-17
12194,-0.728539,-0.519194,2.309871e-16,-7.026354e-16,1.709765e-16,6.117893e-17,3.942038e-16,2.364499e-16,-1.268034e-16,-2.018033e-16
12195,-0.728996,-0.519337,7.500351e-17,-3.911578e-16,-1.953569e-16,1.320182e-16,-1.568254e-16,5.852441e-17,-5.138499e-17,2.330714e-16
12715,-0.726479,-0.518587,-3.522102e-16,-6.045116e-16,-1.106209e-16,-2.994644e-16,1.912922e-16,3.395554e-17,-1.752029e-16,2.543443e-16
12716,-0.727015,-0.518759,-4.717732e-16,-7.037698e-16,9.699056e-18,-9.873998e-17,3.565802e-16,-1.093885e-16,-4.638878e-16,-2.513900e-16
...,...,...,...,...,...,...,...,...,...,...
147437,0.670138,-0.095630,8.080205e-16,3.239531e-16,-1.328647e-16,-5.274026e-17,-2.919215e-16,1.504479e-16,1.771275e-17,1.884973e-16
147438,0.670334,-0.095393,5.399568e-16,3.023191e-16,-8.444971e-18,-1.546697e-16,3.221030e-16,6.387091e-17,2.677115e-16,-3.173276e-18
147439,0.670534,-0.095154,5.796134e-16,2.550234e-16,-4.069585e-16,4.468717e-16,-9.895849e-17,2.081119e-16,2.818884e-16,1.682523e-16
147440,0.670739,-0.094914,2.196856e-16,2.638867e-16,2.946317e-16,3.222806e-16,6.153903e-17,-1.001182e-16,1.544060e-16,-1.498540e-16


In [68]:
preproc.save_obj(area3a_int_df, "area3a_int_df")

## Area 3c

In [69]:
area3c_int = pd.read_excel("../_CIPW/CIPW/AREA3c/interpolated_data.xlsx", index_col=0)
area3c_int = area3c_int.dropna()
area3c_int.drop(["oth", ], axis = 1, inplace = True)
area3c_int = preproc.replace_zero(area3c_int, 0.01)

In [70]:
area3c_int.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51386 entries, 5341 to 97326
Data columns (total 10 columns):
SiO2     51386 non-null float64
TiO2     51386 non-null float64
Al2O3    51386 non-null float64
Fe2O3    51386 non-null float64
FeO      51386 non-null float64
MnO      51386 non-null float64
MgO      51386 non-null float64
CaO      51386 non-null float64
Na2O     51386 non-null float64
K2O      51386 non-null float64
dtypes: float64(10)
memory usage: 4.3 MB


In [71]:
area3c_int

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O
5341,66.538769,0.353725,17.352983,0.058851,3.225805,0.050796,1.244777,4.061044,3.431298,2.939395
5641,66.559359,0.353434,17.345827,0.058987,3.222124,0.050784,1.241623,4.051525,3.431816,2.942355
5642,66.548790,0.353593,17.349408,0.058925,3.224073,0.050792,1.243278,4.056383,3.431536,2.940848
5643,66.537865,0.353756,17.353124,0.058860,3.226076,0.050800,1.244982,4.061414,3.431249,2.939288
5644,66.526592,0.353922,17.356973,0.058791,3.228130,0.050808,1.246735,4.066612,3.430955,2.937677
...,...,...,...,...,...,...,...,...,...,...
97322,59.371784,0.679991,17.089243,0.330165,5.752584,0.096114,3.857076,6.535901,2.969418,2.342484
97323,59.371774,0.679988,17.089269,0.330156,5.752571,0.096114,3.857065,6.535924,2.969421,2.342480
97324,59.371827,0.679984,17.089288,0.330148,5.752546,0.096113,3.857033,6.535917,2.969425,2.342482
97325,59.371943,0.679979,17.089299,0.330141,5.752507,0.096112,3.856981,6.535881,2.969432,2.342490


In [72]:
area3c_int_clr = preproc.clr(area3c_int)
area3c_int_clr.head()

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O
5341,3.667549,-1.569471,2.323529,-3.362983,0.640947,-3.510170,-0.311279,0.871205,0.702703,0.547969
5641,3.668230,-1.569922,2.323488,-3.360299,0.640177,-3.510032,-0.313445,0.869230,0.703226,0.549347
5642,3.667857,-1.569687,2.323480,-3.361561,0.640567,-3.510091,-0.312327,0.870213,0.702929,0.548619
5643,3.667476,-1.569444,2.323477,-3.362887,0.640971,-3.510154,-0.311174,0.871236,0.702629,0.547872
5644,3.667087,-1.569194,2.323480,-3.364279,0.641389,-3.510222,-0.309986,0.872296,0.702324,0.547105


In [73]:
area3c_int_pca = preproc.pca(area3c_int_clr)
preproc.pca_variance(area3c_int_pca)

2 PCA components  out of 10 components with variance sum 1.0 needed for obtaining sum of variance > 0.95


array([7.82116114e-01, 2.17883886e-01, 1.00150527e-30, 1.31754652e-31,
       1.12559396e-31, 8.25804483e-32, 6.08396642e-32, 3.34563593e-32,
       2.85329844e-32, 2.57001574e-32])

In [74]:
area3c_int_df = preproc.create_pca_df(area3c_int_pca, area3c_int_clr)

In [75]:
area3c_int_df

,PC01,PC02,PC03,PC04,PC05,PC06,PC07,PC08,PC09,PC10
5341,1.161454,-0.861487,-6.040973e-16,2.922563e-17,-1.414365e-16,4.606270e-16,-2.665698e-16,1.069774e-17,-6.238805e-17,-1.818885e-16
5641,1.157301,-0.862900,-6.055621e-16,-1.339559e-16,-3.051644e-16,-5.523899e-17,-2.161468e-16,-1.408726e-17,2.001121e-16,1.391180e-16
5642,1.159331,-0.862092,-7.804212e-16,-3.458961e-16,-1.629761e-16,1.336749e-16,-1.944046e-16,3.523863e-16,3.831746e-17,6.430105e-17
5643,1.161449,-0.861274,-7.690333e-16,2.839397e-16,-5.643579e-16,-2.001311e-16,-4.562069e-16,-3.889729e-16,-2.020235e-16,-2.218424e-16
5644,1.163653,-0.860446,-7.623459e-16,-3.873386e-16,-3.359870e-16,1.397440e-17,-1.724508e-16,-1.892676e-17,1.340868e-16,9.096662e-17
...,...,...,...,...,...,...,...,...,...,...
97322,0.530566,0.906701,-9.150829e-16,3.206452e-17,-3.073453e-16,2.194553e-16,4.103908e-17,6.611048e-17,-1.374991e-16,-1.875821e-17
97323,0.530589,0.906687,-9.771044e-16,-5.909757e-16,-1.845935e-16,3.653942e-16,3.773919e-16,4.310806e-17,4.696541e-17,-6.336398e-17
97324,0.530604,0.906669,-1.016789e-15,2.906649e-16,-8.795149e-17,2.225086e-16,-2.230685e-16,9.443359e-17,1.597076e-16,1.770278e-17
97325,0.530612,0.906647,-9.103115e-16,-7.620131e-17,-6.186963e-17,4.628981e-17,1.620876e-16,-5.433015e-17,9.163983e-17,-1.454530e-16


In [76]:
preproc.save_obj(area3c_int_df, "area3c_int_df")

## Area 1a

In [77]:
area1a_int = pd.read_excel("../_CIPW/CIPW/AREA1a/interpolated_data.xlsx", index_col=0)
area1a_int = area1a_int.dropna()
area1a_int.drop(["oth", ], axis = 1, inplace = True)
area1a_int = preproc.replace_zero(area1a_int, 0.01)

In [78]:
area1a_int.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43563 entries, 3952 to 106862
Data columns (total 10 columns):
SiO2     43563 non-null float64
TiO2     43563 non-null float64
Al2O3    43563 non-null float64
Fe2O3    43563 non-null float64
FeO      43563 non-null float64
MnO      43563 non-null float64
MgO      43563 non-null float64
CaO      43563 non-null float64
Na2O     43563 non-null float64
K2O      43563 non-null float64
dtypes: float64(10)
memory usage: 3.7 MB


In [79]:
area1a_int

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O
3952,71.032483,0.258358,14.726474,0.868669,2.068934,0.053045,0.746041,1.658245,4.331207,3.814880
3953,71.032956,0.258313,14.726171,0.868901,2.069092,0.053068,0.745888,1.657975,4.331256,3.814939
3954,71.033447,0.258267,14.725869,0.869124,2.069234,0.053091,0.745732,1.657703,4.331302,3.815002
3955,71.033954,0.258223,14.725570,0.869339,2.069358,0.053113,0.745577,1.657429,4.331347,3.815069
3956,71.034472,0.258180,14.725276,0.869542,2.069464,0.053133,0.745421,1.657157,4.331390,3.815139
...,...,...,...,...,...,...,...,...,...,...
106858,71.476331,0.244759,14.651699,0.881807,1.836063,0.046792,0.663220,1.507454,4.327145,3.896411
106859,71.474066,0.245009,14.652483,0.879631,1.836795,0.046699,0.664023,1.508901,4.326845,3.896102
106860,71.471867,0.245254,14.653256,0.877519,1.837476,0.046607,0.664804,1.510308,4.326551,3.895803
106861,71.469746,0.245491,14.654014,0.875482,1.838100,0.046517,0.665559,1.511667,4.326265,3.895518


In [80]:
area1a_int_clr = preproc.clr(area1a_int)
area1a_int_clr.head()

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O
3952,3.636483,-1.980063,2.062993,-0.767447,0.100380,-3.563272,-0.919628,-0.120894,0.839192,0.712255
3953,3.636464,-1.980265,2.062947,-0.767206,0.100430,-3.562851,-0.919860,-0.121082,0.839178,0.712245
3954,3.636449,-1.980461,2.062904,-0.766971,0.100477,-3.562447,-0.920090,-0.121269,0.839167,0.712240
3955,3.636439,-1.980651,2.062866,-0.766742,0.100519,-3.562060,-0.920317,-0.121452,0.839159,0.712239
3956,3.636432,-1.980833,2.062832,-0.766522,0.100556,-3.561695,-0.920539,-0.121630,0.839155,0.712244


In [81]:
area1a_int_pca = preproc.pca(area1a_int_clr)
preproc.pca_variance(area1a_int_pca)

3 PCA components  out of 10 components with variance sum 1.0 needed for obtaining sum of variance > 0.95


array([7.50802836e-01, 1.97902874e-01, 5.12942905e-02, 5.63741042e-31,
       1.50390716e-31, 1.16087710e-31, 1.02078558e-31, 7.40693515e-32,
       5.13417059e-32, 3.01792668e-32])

In [82]:
area1a_int_df = preproc.create_pca_df(area1a_int_pca, area1a_int_clr)

In [83]:
area1a_int_df

,PC01,PC02,PC03,PC04,PC05,PC06,PC07,PC08,PC09,PC10
3952,-0.375048,0.048924,-0.157410,6.669564e-16,3.387133e-16,-9.658236e-17,7.749990e-17,-2.052048e-16,-7.729073e-18,-2.129473e-16
3953,-0.375202,0.048647,-0.157928,5.628906e-16,-3.586486e-16,-1.902956e-16,-9.448001e-17,-1.680472e-17,-5.133256e-17,-9.967989e-17
3954,-0.375359,0.048372,-0.158428,9.307446e-16,-2.892943e-16,5.188473e-16,-1.165275e-16,5.310360e-17,-6.775892e-17,-2.806662e-17
3955,-0.375516,0.048099,-0.158904,3.731902e-16,5.710048e-17,-8.660261e-17,-1.361177e-16,-4.675298e-16,-1.323030e-16,-8.128159e-17
3956,-0.375672,0.047831,-0.159354,8.217052e-16,-3.546817e-16,-5.283380e-16,-1.477604e-16,-1.800473e-16,8.219640e-17,-3.636863e-16
...,...,...,...,...,...,...,...,...,...,...
106858,-0.442842,-0.093954,-0.062743,5.553748e-16,-5.195019e-17,-3.253328e-16,-1.389341e-16,2.434695e-16,-2.448467e-16,1.027683e-16
106859,-0.442558,-0.091236,-0.060272,2.456002e-16,5.991976e-17,-6.724521e-16,6.108459e-17,7.147994e-17,-1.610087e-16,-1.145576e-16
106860,-0.442276,-0.088602,-0.057838,5.992724e-16,-1.672469e-16,-9.255618e-17,-1.876884e-16,-2.353206e-16,-2.410215e-16,-1.406274e-16
106861,-0.441999,-0.086065,-0.055457,6.323999e-16,-2.033800e-16,-1.091073e-16,8.461433e-17,-7.786667e-17,6.344505e-19,-3.493481e-17


In [84]:
preproc.save_obj(area1a_int_df, "area1a_int_df")

----

# Feature engineering individual areas

## Area1

### Fe_Number

In [85]:
area1_int = pd.read_excel("../_CIPW/CIPW/AREA1/interpolated_data.xlsx", index_col=0)
area1_int = area1_int.dropna()
area1_int.drop(["oth", ], axis = 1, inplace = True)
area1_int = preproc.replace_zero(area1_int, 0.01)
area1_fe1 = pd.DataFrame(index=area1_int.index)
area1_fe = pd.DataFrame(index=area1_int.index)

In [86]:
area1_fe

""
2809
2810
2811
2812
2813
...
205097
205541
205542
205543


In [87]:
area1_fe1['Fe'] = area1_int_clr['FeO'] + area1_int_clr['Fe2O3'] 
area1_fe1['Fe+Mg']= area1_int_clr['MgO'] +area1_int_clr['FeO'] + area1_int_clr['Fe2O3']

In [88]:
area1_fe_x1 = area1_fe1.iloc[:,:1]
area1_fe_x2 = area1_fe1.iloc[:,1:2]

In [89]:
area1_fe1

,Fe,Fe+Mg
2809,-0.953265,-1.526015
2810,-0.952981,-1.527883
2811,-0.952812,-1.529369
2812,-0.968339,-1.576602
2813,-0.968490,-1.577100
...,...,...
205097,-0.650127,-1.689553
205541,-0.644497,-1.682762
205542,-0.645173,-1.683520
205543,-0.645906,-1.684399


In [90]:
scaler1 = StandardScaler().fit(area1_fe_x1)
scaler2 = StandardScaler().fit(area1_fe_x2)

In [91]:
area1_fe_x1_scaled = scaler1.transform(area1_fe_x1)
area1_fe_x2_scaled = scaler2.transform(area1_fe_x2)

In [92]:
area1_fe_x1_scaled

array([[-0.7363528 ],
       [-0.73530557],
       [-0.73468186],
       ...,
       [ 0.40009176],
       [ 0.39738538],
       [ 0.3945772 ]])

In [93]:
area1_fe['Fe'] = area1_fe_x1_scaled
area1_fe['Fe+Mg']= area1_fe_x2_scaled

In [94]:
area1_fe

,Fe,Fe+Mg
2809,-0.736353,0.885658
2810,-0.735306,0.880601
2811,-0.734682,0.876581
2812,-0.791956,0.748738
2813,-0.792515,0.747390
...,...,...
205097,0.381818,0.443018
205541,0.402582,0.461399
205542,0.400092,0.459346
205543,0.397385,0.456967


In [95]:
preproc.save_obj(area1_fe, "area1_fe")

In [96]:
area1_fe_pca = preproc.pca(area1_fe)
preproc.pca_variance(area1_fe_pca)

2 PCA components  out of 2 components with variance sum 1.0 needed for obtaining sum of variance > 0.95


array([0.72795581, 0.27204419])

In [97]:
area1_fe_pca_df = preproc.create_pca_df(area1_fe_pca, area1_fe)

In [98]:
preproc.save_obj(area1_fe_pca_df, "area1_fe_pca_df")

### Mali

In [99]:
area1_int = pd.read_excel("../_CIPW/CIPW/AREA1/interpolated_data.xlsx", index_col=0)
area1_int = area1_int.dropna()
area1_int.drop(["oth", ], axis = 1, inplace = True)
area1_int = preproc.replace_zero(area1_int, 0.01)
area1_MALI1 = pd.DataFrame(index=area1_int.index)
area1_MALI = pd.DataFrame(index=area1_int.index)

In [100]:
area1_MALI1['Na2O'] = area1_int_clr['Na2O']
area1_MALI1['K2O'] = area1_int_clr['K2O']
area1_MALI1['CaO'] = area1_int_clr['CaO']
area1_MALI1['SiO2'] = area1_int_clr['SiO2']

In [101]:
area1_MALI1_x1 = area1_MALI1.iloc[:,:1]
area1_MALI1_x2 = area1_MALI1.iloc[:,1:2]
area1_MALI1_x3 = area1_MALI1.iloc[:,2:3]
area1_MALI1_x4 = area1_MALI1.iloc[:,3:4]

In [102]:
area1_MALI1

,Na2O,K2O,CaO,SiO2
2809,0.719515,0.685173,0.110596,3.617717
2810,0.720766,0.687057,0.108421,3.619309
2811,0.721736,0.688514,0.106766,3.620542
2812,0.719065,0.686071,0.100003,3.607553
2813,0.719245,0.686340,0.099758,3.607763
...,...,...,...,...
205097,0.830836,0.873891,-0.345381,3.706298
205541,0.829211,0.871673,-0.346073,3.704137
205542,0.829395,0.871921,-0.345954,3.704392
205543,0.829613,0.872219,-0.345866,3.704686


In [103]:
scaler1 = StandardScaler().fit(area1_MALI1_x1)
scaler2 = StandardScaler().fit(area1_MALI1_x2)
scaler3 = StandardScaler().fit(area1_MALI1_x3)
scaler4 = StandardScaler().fit(area1_MALI1_x4)

In [104]:
area1_MALI1_x1_scaled = scaler1.transform(area1_MALI1_x1)
area1_MALI1_x2_scaled = scaler2.transform(area1_MALI1_x2)
area1_MALI1_x3_scaled = scaler3.transform(area1_MALI1_x3)
area1_MALI1_x4_scaled = scaler4.transform(area1_MALI1_x4)

In [105]:
area1_MALI['Na2O'] = area1_MALI1_x1_scaled
area1_MALI['K2O'] = area1_MALI1_x2_scaled
area1_MALI['CaO'] = area1_MALI1_x3_scaled
area1_MALI['SiO2'] = area1_MALI1_x4_scaled

In [106]:
area1_MALI

,Na2O,K2O,CaO,SiO2
2809,-1.073343,-1.136156,1.473134,-0.878821
2810,-1.065312,-1.128178,1.466500,-0.870220
2811,-1.059090,-1.122005,1.461450,-0.863553
2812,-1.076231,-1.132352,1.440816,-0.933768
2813,-1.075074,-1.131213,1.440069,-0.932629
...,...,...,...,...
205097,-0.358843,-0.336782,0.081959,-0.399989
205541,-0.369273,-0.346178,0.079847,-0.411673
205542,-0.368091,-0.345127,0.080211,-0.410295
205543,-0.366691,-0.343866,0.080480,-0.408706


In [107]:
preproc.save_obj(area1_MALI, "area1_MALI")

In [108]:
area1_MALI_pca = preproc.pca(area1_MALI)
preproc.pca_variance(area1_MALI_pca)

2 PCA components  out of 4 components with variance sum 0.999526694262567 needed for obtaining sum of variance > 0.95


array([9.27400441e-01, 7.21262536e-02, 4.73305737e-04, 2.09108872e-31])

In [109]:
area1_MALI_pca_df = preproc.create_pca_df(area1_MALI_pca, area1_MALI)

In [110]:
preproc.save_obj(area1_MALI_pca_df, "area1_MALI_pca_df")

## Area3

### Fe_Number

In [111]:
area3_int = pd.read_excel("../_CIPW/CIPW/AREA3/interpolated_data.xlsx", index_col=0)
area3_int = area3_int.dropna()
area3_int.drop(["oth", ], axis = 1, inplace = True)
area3_int = preproc.replace_zero(area3_int, 0.01)
area3_fe1 = pd.DataFrame(index=area3_int.index)
area3_fe = pd.DataFrame(index=area3_int.index)

In [112]:
area3_int

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O
11189,68.973826,0.384729,15.490616,2.105048,2.125391,0.063841,1.003265,2.264021,3.407279,3.337133
11190,68.933533,0.386681,15.498923,2.110996,2.133132,0.064013,1.010837,2.276947,3.406195,3.331904
11191,68.893888,0.388613,15.507131,2.116467,2.140859,0.064183,1.018360,2.289786,3.405126,3.326788
11192,68.856651,0.390440,15.514863,2.121303,2.148205,0.064343,1.025487,2.301946,3.404120,3.322008
11596,69.065490,0.380060,15.469415,2.099817,2.105257,0.063444,0.985195,2.232583,3.409713,3.348840
...,...,...,...,...,...,...,...,...,...,...
160540,65.844594,0.575363,16.199450,1.031606,3.315901,0.073653,1.849313,3.758044,3.304477,3.066352
160541,65.888449,0.572938,16.194183,1.038694,3.300992,0.073544,1.837354,3.738182,3.306089,3.069814
160542,65.963747,0.568772,16.185019,1.050153,3.275856,0.073351,1.816909,3.704308,3.308833,3.075885
160543,65.954892,0.569303,16.186553,1.042429,3.282323,0.073342,1.819655,3.709889,3.308405,3.075937


In [113]:
area3_fe1['Fe'] = area3_int_clr['FeO'] + area3_int_clr['Fe2O3'] 
area3_fe1['Fe+Mg']= area3_int_clr['MgO'] +area3_int_clr['FeO'] + area3_int_clr['Fe2O3']

In [114]:
area3_fe_x1 = area3_fe1.iloc[:,:1]
area3_fe_x2 = area3_fe1.iloc[:,1:2]

In [115]:
scaler1 = StandardScaler().fit(area3_fe_x1)
scaler2 = StandardScaler().fit(area3_fe_x2)

In [116]:
area3_fe_x1_scaled = scaler1.transform(area3_fe_x1)
area3_fe_x2_scaled = scaler2.transform(area3_fe_x2)

In [117]:
area3_fe['Fe'] = area3_fe_x1_scaled
area3_fe['Fe+Mg']= area3_fe_x2_scaled

In [118]:
area3_fe

,Fe,Fe+Mg
11189,1.737837,1.641373
11190,1.741300,1.654477
11191,1.744380,1.667161
11192,1.747016,1.678904
11596,1.737017,1.615164
...,...,...
160540,0.400783,1.557698
160541,0.413489,1.557310
160542,0.433972,1.555632
160543,0.421056,1.548633


In [119]:
preproc.save_obj(area3_fe, "area3_fe")

In [120]:
area3_fe_pca = preproc.pca(area3_fe)
preproc.pca_variance(area3_fe_pca)

2 PCA components  out of 2 components with variance sum 1.0000000000000002 needed for obtaining sum of variance > 0.95


array([0.85776228, 0.14223772])

In [121]:
area3_fe_pca_df = preproc.create_pca_df(area3_fe_pca, area3_fe)

In [122]:
preproc.save_obj(area3_fe_pca_df, "area3_fe_pca_df")

### Mali

In [123]:
area3_int = pd.read_excel("../_CIPW/CIPW/AREA3/interpolated_data.xlsx", index_col=0)
area3_int = area3_int.dropna()
area3_int.drop(["oth", ], axis = 1, inplace = True)
area3_int = preproc.replace_zero(area3_int, 0.01)
area3_MALI1 = pd.DataFrame(index=area3_int.index)
area3_MALI = pd.DataFrame(index=area3_int.index)

In [124]:
area3_MALI1['Na2O'] = area3_int_clr['Na2O']
area3_MALI1['K2O'] = area3_int_clr['K2O']
area3_MALI1['CaO'] = area3_int_clr['CaO']
area3_MALI1['SiO2'] = area3_int_clr['SiO2']

In [125]:
area3_MALI1_x1 = area3_MALI1.iloc[:,:1]
area3_MALI1_x2 = area3_MALI1.iloc[:,1:2]
area3_MALI1_x3 = area3_MALI1.iloc[:,2:3]
area3_MALI1_x4 = area3_MALI1.iloc[:,3:4]

In [126]:
area3_MALI1

,Na2O,K2O,CaO,SiO2
11189,0.424204,0.403402,0.015432,3.432017
11190,0.421336,0.399284,0.018575,3.428883
11191,0.418524,0.395249,0.021700,3.425809
11192,0.415891,0.391474,0.024659,3.422931
11596,0.430760,0.412747,0.007292,3.439188
...,...,...,...,...
160540,0.265732,0.190943,0.394353,3.257751
160541,0.267539,0.193391,0.390373,3.259737
160542,0.270720,0.197717,0.383621,3.263230
160543,0.270764,0.197907,0.385300,3.263269


In [127]:
scaler1 = StandardScaler().fit(area3_MALI1_x1)
scaler2 = StandardScaler().fit(area3_MALI1_x2)
scaler3 = StandardScaler().fit(area3_MALI1_x3)
scaler4 = StandardScaler().fit(area3_MALI1_x4)

In [128]:
area3_MALI1_x1_scaled = scaler1.transform(area3_MALI1_x1)
area3_MALI1_x2_scaled = scaler2.transform(area3_MALI1_x2)
area3_MALI1_x3_scaled = scaler3.transform(area3_MALI1_x3)
area3_MALI1_x4_scaled = scaler4.transform(area3_MALI1_x4)

In [129]:
area3_MALI['Na2O'] = area3_MALI1_x1_scaled
area3_MALI['K2O'] = area3_MALI1_x2_scaled
area3_MALI['CaO'] = area3_MALI1_x3_scaled
area3_MALI['SiO2'] = area3_MALI1_x4_scaled

In [130]:
area3_MALI

,Na2O,K2O,CaO,SiO2
11189,-0.999618,-1.054141,-0.008303,-0.994430
11190,-1.015206,-1.069698,0.005036,-1.010021
11191,-1.030493,-1.084942,0.018296,-1.025311
11192,-1.044805,-1.099205,0.030852,-1.039626
11596,-0.963975,-1.018835,-0.042846,-0.958759
...,...,...,...,...
160540,-1.861041,-1.856831,1.599648,-1.861312
160541,-1.851218,-1.847584,1.582760,-1.851436
160542,-1.833930,-1.831237,1.554108,-1.834060
160543,-1.833691,-1.830519,1.561232,-1.833866


In [131]:
preproc.save_obj(area3_MALI, "area3_MALI")

In [132]:
area3_MALI_pca = preproc.pca(area3_MALI)
preproc.pca_variance(area3_MALI_pca)

2 PCA components  out of 4 components with variance sum 1.0 needed for obtaining sum of variance > 0.95


array([9.37167206e-01, 6.28327944e-02, 5.34505099e-31, 1.25584111e-31])

In [133]:
area3_MALI_pca_df = preproc.create_pca_df(area3_MALI_pca, area3_MALI)

In [134]:
preproc.save_obj(area3_MALI_pca_df, "area3_MALI_pca_df")

## Area3a

### Fe_Number

In [135]:
area3a_int = pd.read_excel("../_CIPW/CIPW/AREA3a/interpolated_data.xlsx", index_col=0)
area3a_int = area3a_int.dropna()
area3a_int.drop(["oth", ], axis = 1, inplace = True)
area3a_int = preproc.replace_zero(area3a_int, 0.01)
area3a_fe1 = pd.DataFrame(index=area3a_int.index)
area3a_fe = pd.DataFrame(index=area3a_int.index)

In [136]:
area3a_int

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O
12193,72.015465,0.290437,14.495370,0.653807,1.400530,0.042271,0.623182,2.141107,3.352642,4.377819
12194,72.017364,0.290373,14.494655,0.653676,1.400141,0.042262,0.622907,2.140542,3.352723,4.378092
12195,72.019207,0.290311,14.493962,0.653548,1.399763,0.042253,0.622639,2.139993,3.352802,4.378356
12715,72.009125,0.290650,14.497747,0.654268,1.401824,0.042302,0.624110,2.142987,3.352371,4.376903
12716,72.011297,0.290577,14.496929,0.654120,1.401378,0.042291,0.623795,2.142340,3.352463,4.377215
...,...,...,...,...,...,...,...,...,...,...
147437,64.586536,0.539193,16.254790,1.161485,3.083182,0.078235,2.242151,4.524845,3.026354,3.529234
147438,64.584667,0.539254,16.255089,1.161418,3.083717,0.078247,2.242595,4.525563,3.026275,3.529077
147439,64.582764,0.539316,16.255394,1.161352,3.084261,0.078259,2.243050,4.526294,3.026195,3.528918
147440,64.580831,0.539379,16.255702,1.161289,3.084812,0.078271,2.243512,4.527035,3.026114,3.528756


In [137]:
area3a_fe1['Fe'] = area3a_int_clr['FeO'] + area3a_int_clr['Fe2O3'] 
area3a_fe1['Fe+Mg']= area3a_int_clr['MgO'] +area3a_int_clr['FeO'] + area3a_int_clr['Fe2O3']

In [138]:
area3a_fe_x1 = area3a_fe1.iloc[:,:1]
area3a_fe_x2 = area3a_fe1.iloc[:,1:2]

In [139]:
scaler1 = StandardScaler().fit(area3a_fe_x1)
scaler2 = StandardScaler().fit(area3a_fe_x2)

In [140]:
area3a_fe_x1_scaled = scaler1.transform(area3a_fe_x1)
area3a_fe_x2_scaled = scaler2.transform(area3a_fe_x2)

In [141]:
area3a_fe['Fe'] = area3a_fe_x1_scaled
area3a_fe['Fe+Mg']= area3a_fe_x2_scaled

In [142]:
area3a_fe

,Fe,Fe+Mg
12193,-0.356125,-0.904279
12194,-0.356631,-0.904982
12195,-0.357124,-0.905665
12715,-0.354361,-0.901883
12716,-0.354932,-0.902681
...,...,...
147437,1.167489,1.191945
147438,1.167442,1.192127
147439,1.167399,1.192315
147440,1.167361,1.192510


In [143]:
preproc.save_obj(area3a_fe, "area3a_fe")

In [144]:
area3a_fe_pca = preproc.pca(area3a_fe)
preproc.pca_variance(area3a_fe_pca)

1 PCA components  out of 2 components with variance sum 0.9809918706493918 needed for obtaining sum of variance > 0.95


array([0.98099187, 0.01900813])

In [145]:
area3a_fe_pca_df = preproc.create_pca_df(area3a_fe_pca, area3a_fe)

In [146]:
preproc.save_obj(area3a_fe_pca_df, "area3a_fe_pca_df")

### Mali

In [147]:
area3a_int = pd.read_excel("../_CIPW/CIPW/AREA3a/interpolated_data.xlsx", index_col=0)
area3a_int = area3a_int.dropna()
area3a_int.drop(["oth", ], axis = 1, inplace = True)
area3a_int = preproc.replace_zero(area3a_int, 0.01)
area3a_MALI1 = pd.DataFrame(index=area3a_int.index)
area3a_MALI = pd.DataFrame(index=area3a_int.index)

In [148]:
area3a_MALI1['Na2O'] = area3a_int_clr['Na2O']
area3a_MALI1['K2O'] = area3a_int_clr['K2O']
area3a_MALI1['CaO'] = area3a_int_clr['CaO']
area3a_MALI1['SiO2'] = area3a_int_clr['SiO2']

In [149]:
area3a_MALI1_x1 = area3a_MALI1.iloc[:,:1]
area3a_MALI1_x2 = area3a_MALI1.iloc[:,1:2]
area3a_MALI1_x3 = area3a_MALI1.iloc[:,2:3]
area3a_MALI1_x4 = area3a_MALI1.iloc[:,3:4]

In [150]:
area3a_MALI1

,Na2O,K2O,CaO,SiO2
12193,0.666018,0.932820,0.217592,3.733150
12194,0.666198,0.933038,0.217484,3.733332
12195,0.666373,0.933250,0.217379,3.733509
12715,0.665413,0.932087,0.217946,3.732538
12716,0.665619,0.932336,0.217822,3.732746
...,...,...,...,...
147437,0.132179,0.285901,0.534404,3.192826
147438,0.132087,0.285791,0.534497,3.192732
147439,0.131994,0.285679,0.534591,3.192635
147440,0.131899,0.285565,0.534687,3.192537


In [151]:
scaler1 = StandardScaler().fit(area3a_MALI1_x1)
scaler2 = StandardScaler().fit(area3a_MALI1_x2)
scaler3 = StandardScaler().fit(area3a_MALI1_x3)
scaler4 = StandardScaler().fit(area3a_MALI1_x4)

In [152]:
area3a_MALI1_x1_scaled = scaler1.transform(area3a_MALI1_x1)
area3a_MALI1_x2_scaled = scaler2.transform(area3a_MALI1_x2)
area3a_MALI1_x3_scaled = scaler3.transform(area3a_MALI1_x3)
area3a_MALI1_x4_scaled = scaler4.transform(area3a_MALI1_x4)

In [153]:
area3a_MALI['Na2O'] = area3a_MALI1_x1_scaled
area3a_MALI['K2O'] = area3a_MALI1_x2_scaled
area3a_MALI['CaO'] = area3a_MALI1_x3_scaled
area3a_MALI['SiO2'] = area3a_MALI1_x4_scaled

In [154]:
area3a_MALI

,Na2O,K2O,CaO,SiO2
12193,1.480318,1.468391,-2.070530,1.504653
12194,1.481188,1.469258,-2.071436,1.505529
12195,1.482033,1.470101,-2.072313,1.506380
12715,1.477397,1.465479,-2.067564,1.501715
12716,1.478391,1.466469,-2.068603,1.502715
...,...,...,...,...
147437,-1.098104,-1.101860,0.583984,-1.090115
147438,-1.098546,-1.102297,0.584765,-1.090570
147439,-1.098998,-1.102743,0.585556,-1.091033
147440,-1.099458,-1.103197,0.586356,-1.091505


In [155]:
preproc.save_obj(area3a_MALI, "area3a_MALI")

In [156]:
area3a_MALI_pca = preproc.pca(area3a_MALI)
preproc.pca_variance(area3a_MALI_pca)

2 PCA components  out of 4 components with variance sum 0.9999999999999999 needed for obtaining sum of variance > 0.95


array([9.25160830e-01, 7.48391696e-02, 4.72933727e-31, 1.37311408e-31])

In [157]:
area3a_MALI_pca_df = preproc.create_pca_df(area3a_MALI_pca, area3a_MALI)

In [158]:
preproc.save_obj(area3a_MALI_pca_df, "area3a_MALI_pca_df")

## Area3c

### Fe_Number

In [ ]:
area3c_int = pd.read_excel("../_CIPW/CIPW/AREA3c/interpolated_data.xlsx", index_col=0)
area3c_int = area3c_int.dropna()
area3c_int.drop(["oth", ], axis = 1, inplace = True)
area3c_int = preproc.replace_zero(area3c_int, 0.01)
area3c_fe1 = pd.DataFrame(index=area3c_int.index)
area3c_fe = pd.DataFrame(index=area3c_int.index)

In [ ]:
area3c_int

In [ ]:
area3c_fe1['Fe'] = area3c_int_clr['FeO'] + area3c_int_clr['Fe2O3'] 
area3c_fe1['Fe+Mg']= area3c_int_clr['MgO'] +area3c_int_clr['FeO'] + area3c_int_clr['Fe2O3']

In [ ]:
area3c_fe_x1 = area3c_fe1.iloc[:,:1]
area3c_fe_x2 = area3c_fe1.iloc[:,1:2]

In [ ]:
scaler1 = StandardScaler().fit(area3c_fe_x1)
scaler2 = StandardScaler().fit(area3c_fe_x2)

In [ ]:
area3c_fe_x1_scaled = scaler1.transform(area3c_fe_x1)
area3c_fe_x2_scaled = scaler2.transform(area3c_fe_x2)

In [ ]:
area3c_fe['Fe'] = area3c_fe_x1_scaled
area3c_fe['Fe+Mg']= area3c_fe_x2_scaled

In [ ]:
area3c_fe

In [ ]:
preproc.save_obj(area3c_fe, "area3c_fe")

In [ ]:
area3c_fe_pca = preproc.pca(area3c_fe)
preproc.pca_variance(area3c_fe_pca)

In [ ]:
area3c_fe_pca_df = preproc.create_pca_df(area3c_fe_pca, area3c_fe)

In [ ]:
preproc.save_obj(area3c_fe_pca_df, "area3c_fe_pca_df")

### Mali

In [178]:
area3c_int = pd.read_excel("../_CIPW/CIPW/AREA3c/interpolated_data.xlsx", index_col=0)
area3c_int = area3c_int.dropna()
area3c_int.drop(["oth", ], axis = 1, inplace = True)
area3c_int = preproc.replace_zero(area3c_int, 0.01)
area3c_MALI1 = pd.DataFrame(index=area3c_int.index)
area3c_MALI = pd.DataFrame(index=area3c_int.index)

In [179]:
area3c_MALI1['Na2O'] = area3c_int_clr['Na2O']
area3c_MALI1['K2O'] = area3c_int_clr['K2O']
area3c_MALI1['CaO'] = area3c_int_clr['CaO']
area3c_MALI1['SiO2'] = area3c_int_clr['SiO2']

In [180]:
area3c_MALI1_x1 = area3c_MALI1.iloc[:,:1]
area3c_MALI1_x2 = area3c_MALI1.iloc[:,1:2]
area3c_MALI1_x3 = area3c_MALI1.iloc[:,2:3]
area3c_MALI1_x4 = area3c_MALI1.iloc[:,3:4]

In [181]:
area3c_MALI1

,Na2O,K2O,CaO,SiO2
5341,0.702703,0.547969,0.871205,3.667549
5641,0.703226,0.549347,0.869230,3.668230
5642,0.702929,0.548619,0.870213,3.667857
5643,0.702629,0.547872,0.871236,3.667476
5644,0.702324,0.547105,0.872296,3.667087
...,...,...,...,...
97322,0.088100,-0.149054,0.877044,3.083553
97323,0.088105,-0.149052,0.877052,3.083557
97324,0.088111,-0.149046,0.877055,3.083562
97325,0.088118,-0.149038,0.877055,3.083570


In [182]:
scaler1 = StandardScaler().fit(area3c_MALI1_x1)
scaler2 = StandardScaler().fit(area3c_MALI1_x2)
scaler3 = StandardScaler().fit(area3c_MALI1_x3)
scaler4 = StandardScaler().fit(area3c_MALI1_x4)

In [183]:
area3c_MALI1_x1_scaled = scaler1.transform(area3c_MALI1_x1)
area3c_MALI1_x2_scaled = scaler2.transform(area3c_MALI1_x2)
area3c_MALI1_x3_scaled = scaler3.transform(area3c_MALI1_x3)
area3c_MALI1_x4_scaled = scaler4.transform(area3c_MALI1_x4)

In [184]:
area3c_MALI['Na2O'] = area3c_MALI1_x1_scaled
area3c_MALI['K2O'] = area3c_MALI1_x2_scaled
area3c_MALI['CaO'] = area3c_MALI1_x3_scaled
area3c_MALI['SiO2'] = area3c_MALI1_x4_scaled

In [185]:
area3c_MALI

,Na2O,K2O,CaO,SiO2
5341,2.027553,0.807246,1.061660,1.658533
5641,2.031158,0.813131,1.055810,1.663138
5642,2.029111,0.810025,1.058724,1.660611
5643,2.027036,0.806833,1.061751,1.658033
5644,2.024935,0.803558,1.064891,1.655407
...,...,...,...,...
97322,-2.214879,-2.169129,1.078954,-2.290719
97323,-2.214846,-2.169118,1.078977,-2.290694
97324,-2.214804,-2.169095,1.078987,-2.290656
97325,-2.214752,-2.169058,1.078987,-2.290608


In [186]:
preproc.save_obj(area3c_MALI, "area3c_MALI")

In [187]:
area3c_MALI_pca = preproc.pca(area3c_MALI)
preproc.pca_variance(area3c_MALI_pca)

2 PCA components  out of 4 components with variance sum 1.0000000000000002 needed for obtaining sum of variance > 0.95


array([7.81322096e-01, 2.18677904e-01, 9.08862689e-31, 1.60192741e-31])

In [188]:
area3c_MALI_pca_df = preproc.create_pca_df(area3c_MALI_pca, area3c_MALI)

In [189]:
preproc.save_obj(area3c_MALI_pca_df, "area3c_MALI_pca_df")

# combined areas

## PCA

In [48]:
combined_int_clr = pd.concat([area1_int_clr, area2_int_clr, area3_int_clr, area4_int_clr, area5_int_clr])

In [49]:
combined_int_clr

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O
2809,3.617717,-1.818107,2.076606,-1.061423,0.108158,-3.865485,-0.572750,0.110596,0.719515,0.685173
2810,3.619309,-1.819333,2.077737,-1.060040,0.107059,-3.866074,-0.574903,0.108421,0.720766,0.687057
2811,3.620542,-1.820289,2.078616,-1.059059,0.106247,-3.866515,-0.576557,0.106766,0.721736,0.688514
2812,3.607553,-1.806754,2.063899,-1.122872,0.154534,-3.793236,-0.608263,0.100003,0.719065,0.686071
2813,3.607763,-1.806927,2.064040,-1.123088,0.154597,-3.793120,-0.608609,0.099758,0.719245,0.686340
...,...,...,...,...,...,...,...,...,...,...
123582,3.609088,-1.704755,2.088912,-1.091036,0.086613,-3.650760,-0.794261,0.101916,0.559772,0.794510
123583,3.609082,-1.704739,2.088908,-1.091088,0.086639,-3.650755,-0.794243,0.101928,0.559763,0.794505
123584,3.609079,-1.704727,2.088907,-1.091136,0.086662,-3.650750,-0.794233,0.101938,0.559757,0.794504
123585,3.609079,-1.704720,2.088908,-1.091180,0.086681,-3.650745,-0.794230,0.101945,0.559756,0.794507


In [265]:
combined_int_pca = preproc.pca(combined_int_clr)
preproc.pca_variance(combined_int_pca)

4 PCA components  out of 10 components with variance sum 0.975110434272905 needed for obtaining sum of variance > 0.95


array([5.65992334e-01, 2.92405424e-01, 7.03070614e-02, 4.64056145e-02,
       1.76437665e-02, 3.37164734e-03, 2.96154824e-03, 8.61565268e-04,
       5.10384167e-05, 1.91179517e-31])

In [266]:
combined_int_df = preproc.create_pca_df(combined_int_pca, combined_int_clr)

In [267]:
combined_int_df

,PC01,PC02,PC03,PC04,PC05,PC06,PC07,PC08,PC09,PC10
2809,-0.249795,0.339154,0.335131,0.125243,-0.098494,-0.021777,0.079573,0.055533,-0.001933,3.072091e-16
2810,-0.245177,0.338033,0.335354,0.125711,-0.098888,-0.021665,0.079688,0.055063,-0.001891,6.321045e-16
2811,-0.241632,0.337259,0.335501,0.126088,-0.099184,-0.021569,0.079778,0.054708,-0.001859,1.705823e-16
2812,-0.253641,0.395319,0.242689,0.098330,-0.097995,-0.018807,0.063234,0.069656,-0.002461,3.846682e-16
2813,-0.253084,0.395573,0.242442,0.098385,-0.098026,-0.018759,0.063219,0.069644,-0.002456,6.644959e-16
...,...,...,...,...,...,...,...,...,...,...
123582,-0.155484,0.355480,0.133565,-0.102384,0.077293,-0.050149,-0.009250,-0.021504,-0.003547,-1.359656e-16
123583,-0.155519,0.355533,0.133554,-0.102391,0.077287,-0.050150,-0.009255,-0.021505,-0.003548,-6.964982e-17
123584,-0.155542,0.355583,0.133544,-0.102396,0.077284,-0.050150,-0.009261,-0.021506,-0.003548,-7.398931e-17
123585,-0.155553,0.355630,0.133533,-0.102398,0.077282,-0.050151,-0.009266,-0.021507,-0.003548,1.031985e-16


In [268]:
preproc.save_obj(combined_int_df, "combined_int_df")

----

## Fe_Number

In [316]:
Fe_combined1 = pd.read_excel("../_CLUSTER/groups_time_area/interpolation/combined_int_values.xlsx", index_col=0, usecols = 0)
Fe_combined = pd.read_excel("../_CLUSTER/groups_time_area/interpolation/combined_int_values.xlsx", index_col=0, usecols = 0)


C:\Users\Sebastian\Anaconda3\envs\main\lib\site-packages\pandas\io\excel\_base.py:445: FutureWarning: Passing in an integer for `usecols` has been deprecated. Please pass in a list of int from 0 to `usecols` inclusive instead.
  usecols = _maybe_convert_usecols(usecols)


In [317]:
Fe_combined1['Fe'] = combined_int_clr['FeO'] +  combined_int_clr['Fe2O3']
Fe_combined1['Fe+Mg'] =  combined_int_clr['Fe2O3'] + combined_int_clr['FeO'] + combined_int_clr['MgO']


In [318]:
Fe_combined1

,Fe,Fe+Mg
2809,-0.953265,-1.526015
2810,-0.952981,-1.527883
2811,-0.952812,-1.529369
2812,-0.968339,-1.576602
2813,-0.968490,-1.577100
...,...,...
123582,-1.004423,-1.798684
123583,-1.004449,-1.798692
123584,-1.004474,-1.798708
123585,-1.004499,-1.798730


In [319]:
Fe_comb_x3

""
2809
2810
2811
2812
2813
...
123582
123583
123584
123585


In [320]:
Fe_combined = Fe_combined1

In [321]:
preproc.save_obj(Fe_combined, "Fe_combined")

In [322]:
Fe_combined_pca = preproc.pca(Fe_combined)
preproc.pca_variance(Fe_combined_pca)

2 PCA components  out of 2 components with variance sum 1.0 needed for obtaining sum of variance > 0.95


array([0.88839465, 0.11160535])

In [323]:
Fe_combined_pca_df = preproc.create_pca_df(Fe_combined_pca, Fe_combined)

In [324]:
preproc.save_obj(Fe_combined_pca_df, "Fe_combined_pca_df")

## MALI

In [307]:
MALI_combined1 = pd.read_excel("../_CLUSTER/groups_time_area/interpolation/combined_int_values.xlsx", index_col=0, usecols = 0)
MALI_combined = pd.read_excel("../_CLUSTER/groups_time_area/interpolation/combined_int_values.xlsx", index_col=0, usecols = 0)

C:\Users\Sebastian\Anaconda3\envs\main\lib\site-packages\pandas\io\excel\_base.py:445: FutureWarning: Passing in an integer for `usecols` has been deprecated. Please pass in a list of int from 0 to `usecols` inclusive instead.
  usecols = _maybe_convert_usecols(usecols)


In [308]:
MALI_combined1['Na2O'] = combined_int_clr['Na2O']
MALI_combined1['K2O'] = combined_int_clr['K2O']
MALI_combined1['CaO'] = combined_int_clr['CaO']
#MALI_combined1['SiO2'] = combined_int_clr['SiO2']

In [309]:
MALI_combined_x1 = MALI_combined1.iloc[:,:1]
MALI_combined_x2 = MALI_combined1.iloc[:,1:2]
MALI_combined_x3 = MALI_combined1.iloc[:,2:3]
#MALI_combined_x4 = MALI_combined1.iloc[:,3:4]

In [310]:
MALI_combined1

,Na2O,K2O,CaO
2809,0.719515,0.685173,0.110596
2810,0.720766,0.687057,0.108421
2811,0.721736,0.688514,0.106766
2812,0.719065,0.686071,0.100003
2813,0.719245,0.686340,0.099758
...,...,...,...
123582,0.559772,0.794510,0.101916
123583,0.559763,0.794505,0.101928
123584,0.559757,0.794504,0.101938
123585,0.559756,0.794507,0.101945


In [311]:
MALI_combined = MALI_combined1

In [312]:
preproc.save_obj(MALI_combined, "MALI_combined")

In [313]:
MALI_combined_pca = preproc.pca(MALI_combined)
preproc.pca_variance(MALI_combined_pca)

2 PCA components  out of 3 components with variance sum 0.9588430804568939 needed for obtaining sum of variance > 0.95


array([0.8451536 , 0.11368948, 0.04115692])

In [314]:
MALI_combined_pca_df = preproc.create_pca_df(MALI_combined_pca, MALI_combined)

In [315]:
preproc.save_obj(MALI_combined_pca_df, "MALI_combined_pca_df")